<a href="https://colab.research.google.com/github/sishaque/Anomaly-detection-using-a-sparse-autoencoder-tsne-and-NN/blob/main/Transfer_Learning_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import numpy as np
import pandas as pd
import numpy as np
import cv2
import biosppy
import matplotlib.pyplot as plt
import re
import sys
import os
from keras.preprocessing import image
import wfdb

In [ ]:
pip install wfdb

In [ ]:
pip install biosppy

In [ ]:
df = pd.read_csv("/content/U12T3_dataECG.csv", sep=',', usecols= [2])
df = df*-1
print(df)
df = np.transpose(df)
ECG_data = np.array(df)
ECG_data = ECG_data.flatten()
print(ECG_data)



In [ ]:
def segmentation(records):
    Normal = []
    for e in records:
        signals, fields = wfdb.rdsamp(e, channels = [0]) 

        ann = wfdb.rdann(e, '/content/Saved_dataECG (4).csv')
        good = ['N']
        ids = np.in1d(ann.symbol, good)
        imp_beats = ann.sample[ids]
        beats = (ann.sample)
        for i in imp_beats:
            beats = list(beats)
            j = beats.index(i)
            if(j!=0 and j!=(len(beats)-1)):
                x = beats[j-1]
                y = beats[j+1]
                diff1 = abs(x - beats[j])//2
                diff2 = abs(y - beats[j])//2
                Normal.append(signals[beats[j] - diff1: beats[j] + diff2, 0])
    return Normal

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
def ECG_signal(csv_data):
  data = np.array(csv_data)
  signals = []
  count = 1
  peaks =  biosppy.signals.ecg.christov_segmenter(signal=data, sampling_rate = 200)[0]
  for i in (peaks[1:-1]):
    diff1 = abs(peaks[count - 1] - i)
    diff2 = abs(peaks[count + 1]- i)
    x = peaks[count - 1] + diff1//2
    y = peaks[count + 1] - diff2//2
    signal = data[x:y]
    signals.append(signal)
    count += 1
  return signals

In [ ]:
# data = segmentation(df)
Signal = ECG_signal(ECG_data)

for count, i in enumerate(Signal):
  fig = plt.figure(frameon=False)
  plt.plot(i) 
  plt.xticks([]), plt.yticks([])
  for spine in plt.gca().spines.values():
     spine.set_visible(False)

  filename = '/content/drive/MyDrive/ECG_stress_image_dataset/stressed/' + str(2939+count)+'.png'
  plt.savefig(filename)
  im_gray = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
  im_gray = cv2.resize(im_gray, (128, 128), interpolation = cv2.INTER_LANCZOS4)
  cv2.imwrite(filename, im_gray)

In [ ]:
pip install pigeon-jupyter

In [ ]:
import os
import shutil


not_stressed_dataset_dir = '/content/drive/MyDrive/ECG_stress_image_dataset/not_stressed'
not_stressed_train = os.path.join(not_stressed_dataset_dir, 'not_stressed_train')
# os.mkdir(not_stressed_train)

fnames = ['{}.png'.format(i) for i in range(1800)]
for fname in fnames:
  src = os.path.join(not_stressed_dataset_dir, fname)
  dst = os.path.join(not_stressed_train, fname)
  shutil.copyfile(src,dst)




In [ ]:
import os
import shutil

stressed_dataset_dir = '/content/drive/MyDrive/ECG_stress_image_dataset/stressed'
stress_train_dir = os.path.join('/content/drive/MyDrive/ECG_stress_image_dataset/', 'stress_train_dir')
# os.mkdir(stress_train_dir)

fnames = ['{}.png'.format(i+2001) for i in range(1000)]
for fname in fnames:
  src = os.path.join(stressed_dataset_dir, fname)
  dst = os.path.join(stress_train_dir, fname)
  shutil.copyfile(src,dst)

In [ ]:
stressed_dataset_dir = '/content/drive/MyDrive/ECG_stress_image_dataset/stressed'
stress_test_dir = os.path.join('/content/drive/MyDrive/ECG_stress_image_dataset/', 'stress_test_dir')
# os.mkdir(stress_test_dir)

fnames = ['{}.png'.format(i+501) for i in range(500)]
for fname in fnames:
  src = os.path.join(stressed_dataset_dir, fname)
  dst = os.path.join(stress_test_dir, fname)
  shutil.copyfile(src,dst)

In [ ]:
stress_train_dir = '/content/drive/MyDrive/ECG_stress_image_dataset/stress_train_dir'
stress_val_dir = '/content/drive/MyDrive/ECG_stress_image_dataset/stress_val_dir'

In [ ]:
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt


model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation = 'relu', input_shape = (128, 128, 3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = optimizers.RMSprop(lr = 1e-4), metrics = ['acc'])


train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    stress_train_dir,
    target_size = (128, 128),
    batch_size = 20,
    class_mode = 'binary')




validation_generator = test_datagen.flow_from_directory(
    stress_val_dir,
    target_size = (128, 128),
    batch_size = 20,
    class_mode = 'binary')



for data_batch, labels_batch in train_generator:
  print('data batch shape:', data_batch.shape)
  print('labels batch shape', labels_batch.shape)
  break



history = model.fit_generator(train_generator,
                              steps_per_epoch = 100,
                              epochs = 30,
                              validation_data = validation_generator,
                              validation_steps = 50)





acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label = 'Trainning acc')
plt.plot(epochs, val_acc, 'b', label = 'Validation acc')
plt.title('Trainning and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label = 'Trainning loss')
plt.plot(epochs, val_acc, 'b', label = 'Validation loss')
plt.title('Trainning and validation loss')
plt.legend()
plt.figure()
plt.show()


In [ ]:
pip install pigeon-jupyter

In [ ]:
from pigeon import annotate

In [ ]:
from IPython.display import display, Image

annotations = annotate(
    ['/content/drive/MyDrive/ECG_stress_image_dataset/stress_train_dir/0.png',
     '/content/drive/MyDrive/ECG_stress_image_dataset/stress_train_dir/1.png',
     '/content/drive/MyDrive/ECG_stress_image_dataset/stress_train_dir/2.png',
     '/content/drive/MyDrive/ECG_stress_image_dataset/stress_train_dir/3.png',
     '/content/drive/MyDrive/ECG_stress_image_dataset/stress_train_dir/4.png',
     '/content/drive/MyDrive/ECG_stress_image_dataset/stress_train_dir/5.png',
     '/content/drive/MyDrive/ECG_stress_image_dataset/stress_train_dir/6.png',
     '/content/drive/MyDrive/ECG_stress_image_dataset/stress_train_dir/7.png',
     '/content/drive/MyDrive/ECG_stress_image_dataset/stress_train_dir/8.png',
     '/content/drive/MyDrive/ECG_stress_image_dataset/stress_train_dir/9.png',
     '/content/drive/MyDrive/ECG_stress_image_dataset/stress_train_dir/10.png',
     '/content/drive/MyDrive/ECG_stress_image_dataset/stress_train_dir/11.png',
     '/content/drive/MyDrive/ECG_stress_image_dataset/stress_train_dir/12.png',
     '/content/drive/MyDrive/ECG_stress_image_dataset/stress_train_dir/13.png',
     '/content/drive/MyDrive/ECG_stress_image_dataset/stress_train_dir/14.png',
     '/content/drive/MyDrive/ECG_stress_image_dataset/stress_train_dir/15.png',
     '/content/drive/MyDrive/ECG_stress_image_dataset/stress_train_dir/16.png',
     '/content/drive/MyDrive/ECG_stress_image_dataset/stress_train_dir/17.png',
     '/content/drive/MyDrive/ECG_stress_image_dataset/stress_train_dir/18.png',
     '/content/drive/MyDrive/ECG_stress_image_dataset/stress_train_dir/19.png',
     '/content/drive/MyDrive/ECG_stress_image_dataset/stress_train_dir/20.png',
     '/content/drive/MyDrive/ECG_stress_image_dataset/stress_train_dir/21.png',
     '/content/drive/MyDrive/ECG_stress_image_dataset/stress_train_dir/22.png',
     '/content/drive/MyDrive/ECG_stress_image_dataset/stress_train_dir/23.png'],
    options = ['not stressed', 'stressed' ],
    display_fn = lambda filename: display(Image(filename))
    
)

In [ ]:
for data, label in annotations:
  print("data", data)
  print("label", label)

In [ ]:
annotations

x, y = image.img_to_array(annotations)

print(x)

In [ ]:
import os
import h5py
import wfdb as wf
import numpy as np
import pandas as pd
from glob import glob
from scipy import signal as ss
from utils import download as ud
from matplotlib import pyplot as plt

def get_records():
    """ Get paths for data in data/mit/ directory """
    # Download if doesn't exist
    if not os.path.isdir('data/mitdb'):
        print 'Downloading the mitdb ecg database, please wait'
        ud.download_mitdb()
        print 'Download finished'

    # There are 3 files for each record
    # *.atr is one of them
    paths = glob('data/mitdb/*.atr')

    # Get rid of the extension
    paths = [path[:-4] for path in paths]
    paths.sort()

    return paths

def good_types():
    """ Of annotations """
    # www.physionet.org/physiobank/annotations.shtml
    good = ['N', 'L', 'R', 'B', 'A',
            'a', 'J', 'S', 'V', 'r',
            'F', 'e', 'j', 'n', 'E',
            '/', 'f', 'Q', '?']

    return good

def beat_annotations(annotation):
    """ Get rid of non-beat markers """
    # Declare beat types
    good = good_types()
    ids = np.in1d(annotation.anntype, good)

    # We want to know only the positions
    beats = annotation.annsamp[ids]

    return beats

def convert_input(channel, annotation):
    """ Into output """
    # Remove non-beat annotations
    beats = beat_annotations(annotation)

    # Create dirac-comb signal
    dirac = np.zeros_like(channel)
    dirac[beats] = 1.0

    # Use hamming window as a bell-curve filter
    width = 36
    filter = ss.hamming(width)
    gauss = np.convolve(filter, dirac, mode = 'same')

    return dirac, gauss

def good_annotations():
    """ Const function with good annotations """
    # For now it seems those are most popular
    good_annotations = [1, 2, 3, 4,
                        5, 6, 7, 8,
                        9, 10, 11, 12,
                        13, 16, 31, 38]

    return good_annotations

def make_dataset(records, width, savepath):
    """ Inside an array """
    # Prepare containers
    signals, labels = [], []

    # Iterate files
    for path in records:
        print 'Processing file:', path
        record = wf.rdsamp(path)
        annotations = wf.rdann(path, 'atr')

        # Extract pure signals
        data = record.p_signals

        # Convert each channel into labeled fragments
        signal, label = convert_data(data, annotations, width)

        # Cumulate
        signals.append(signal)
        labels.append(label)

    # Convert to one huge numpy.array
    signals = np.vstack(signals)
    labels = np.vstack(labels)

    # Write to disk
    np.save(savepath, {'signals' : signals,
                       'labels'  : labels })

def convert_data(data, annotations, width):
    """ Into a batch """
    # Prepare containers
    signals, labels = [], []

    # Convert both channels
    for it in range(2):
        channel = data[:, it]
        dirac, gauss = convert_input(channel,
                                     annotations)
        # Merge labels
        label = np.vstack([dirac, gauss])

        # Prepare the moving window
        sta = 0
        end = width
        stride = width
        while end <= len(channel):
            # Chop out the fragments
            s_frag = channel[sta : end]
            l_frag = label[:, sta : end]

            # Cumulate
            signals.append(s_frag)
            labels.append(l_frag)

            # Go forth
            sta += stride
            end += stride

    # Turn into arrays
    signals = np.array(signals)
    labels = np.array(labels)

    return signals, labels

def create_datasets():
    """ Training, validation, test """
    # Prepare paths
    records = get_records()

    # Shuffle up determinitically
    np.random.seed(666)
    np.random.shuffle(records)

    # Define the data
    width = 200

    # Make training
    make_dataset(records[:30], width, 'data/training')

    # ... validation ...
    make_dataset(records[30 : 39], width, 'data/validation')

    # ... and test
    make_dataset(records[39 : 48], width, 'data/test')

Testing the Stress Image Dataset

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
pip install tqdm

In [ ]:
!pip install tensorflow-gpu

In [ ]:
!nvidia-smi

In [ ]:
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray

from mrcnn.config import Config
from mrcnn.model import MaskRCNN

In [ ]:
pip install mrcnn

In [ ]:
import tensorflow 
# from adamod import AdaMod
#loading the training images

x_train = pd.read_csv('/content/drive/MyDrive/Stress_image_detection/data/train.csv')
x_train_img = x_train['filename']


# # load the image for google drive file to train_img
train_image = []
for i in tqdm(x_train_img):
    img = image.load_img('/content/drive/MyDrive/Stress_image_detection/Stress_image_dataset/train/' + str(i), target_size=(128,128,3), grayscale = False)
    img = image.img_to_array(img)
    # img = tf.image.rgb_to_grayscale(img)
    # img = np.expand_dims(img, axis=0)
    img = img/255
    train_image.append(img)



In [ ]:
pip install adamod

In [ ]:
import math
import torch
from torch.optim import Optimizer

class AdaMod(Optimizer):
    """Implements AdaMod algorithm with Decoupled Weight Decay (arxiv.org/abs/1711.05101)
    It has been proposed in `Adaptive and Momental Bounds for Adaptive Learning Rate Methods`_.
    Arguments:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float, optional): learning rate (default: 1e-3)
        betas (Tuple[float, float], optional): coefficients used for computing
            running averages of gradient and its square (default: (0.9, 0.999))
        beta3 (float, optional): smoothing coefficient for adaptive learning rates (default: 0.9999)
        eps (float, optional): term added to the denominator to improve
            numerical stability (default: 1e-8)
        weight_decay (float, optional): weight decay (L2 penalty) (default: 0)
    """

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), beta3=0.999,
                 eps=1e-8, weight_decay=0):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
        if not 0.0 <= beta3 < 1.0:
            raise ValueError("Invalid beta3 parameter: {}".format(beta3))
        defaults = dict(lr=lr, betas=betas, beta3=beta3, eps=eps,
                        weight_decay=weight_decay)
        super(AdaMod, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(AdaMod, self).__setstate__(state)

    def step(self, closure=None):
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError(
                        'AdaMod does not support sparse gradients')

                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    # Exponential moving average of gradient values
                    state['exp_avg'] = torch.zeros_like(p.data)
                    # Exponential moving average of squared gradient values
                    state['exp_avg_sq'] = torch.zeros_like(p.data)
                    # Exponential moving average of actual learning rates
                    state['exp_avg_lr'] = torch.zeros_like(p.data)

                exp_avg, exp_avg_sq, exp_avg_lr = state['exp_avg'], state['exp_avg_sq'], state['exp_avg_lr']
                beta1, beta2 = group['betas']

                state['step'] += 1

                # Decay the first and second moment running average coefficient
                exp_avg.mul_(beta1).add_(1 - beta1, grad)
                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)

                denom = exp_avg_sq.sqrt().add_(group['eps'])

                bias_correction1 = 1 - beta1 ** state['step']
                bias_correction2 = 1 - beta2 ** state['step']
                step_size = group['lr'] * math.sqrt(bias_correction2) / bias_correction1

                if group['weight_decay'] != 0:
                    p.data.add_(-group['weight_decay'] * group['lr'], p.data)

                # Applies momental bounds on actual learning rates
                step_size = torch.full_like(denom, step_size)
                step_size.div_(denom)
                exp_avg_lr.mul_(group['beta3']).add_(1 - group['beta3'], step_size)
                step_size = torch.min(step_size,  exp_avg_lr)
                step_size.mul_(exp_avg)

                p.data.add_(-step_size)

        return loss

In [ ]:
# # converting the images to an array
X_train_img = np.array(train_image)

# initializing the label
# converting the text label to numbers
y_train = []
for i in x_train['class']:
  if i == 'not_stressed':
    i = 0
  else:
    i = 1
    
  y_train.append(i)

# converting the label to categorical
y_train_label = to_categorical(y_train)




In [ ]:
# Train image data augmentation

from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# train_aug

train_aug = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    zoom_range = 0.15,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.15,
    horizontal_flip = True,
    fill_mode = "nearest"

)

# val_aug
val_aug = ImageDataGenerator(
    rotation_range = 20,
    zoom_range = 0.15,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.15,
    horizontal_flip = True,
    fill_mode = "nearest"
    
)






In [ ]:





train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.


train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    stress_train_dir,
    target_size = (128, 128),
    batch_size = 20,
    class_mode = 'binary')




validation_generator = test_datagen.flow_from_directory(
    stress_val_dir,
    target_size = (128, 128),
    batch_size = 20,
    class_mode = 'binary')



for data_batch, labels_batch in train_generator:
  print('data batch shape:', data_batch.shape)
  print('labels batch shape', labels_batch.shape)
  break



history = model.fit_generator(train_generator,
                              steps_per_epoch = 100,
                              epochs = 30,
                              validation_data = validation_generator,
                              validation_steps = 50)





acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label = 'Trainning acc')
plt.plot(epochs, val_acc, 'b', label = 'Validation acc')
plt.title('Trainning and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label = 'Trainning loss')
plt.plot(epochs, val_acc, 'b', label = 'Validation loss')
plt.title('Trainning and validation loss')
plt.legend()
plt.figure()
plt.show()


In [ ]:
# loading validation data and converting to an array

x_val = pd.read_csv('/content/drive/MyDrive/Stress_image_detection/data/valid.csv')
x_val_img = x_val['filename']

val_images = []

for i in tqdm(x_val_img):
  val_img = image.load_img('/content/drive/MyDrive/Stress_image_detection/Stress_image_dataset/valid/' + str(i), target_size=(128, 128, 3), grayscale = False)
  val_img = image.img_to_array(val_img)
  # val_img = tf.image.rgb_to_grayscale(val_img)
  # val_img = np.expand_dims(val_img, axis=0)
  val_img = val_img/255
  val_images.append(val_img)



In [ ]:
# converting image to an array
X_val = np.array(val_images)

# initializing labl
y_val = []

for i in (x_val['class']):
  if i == 'not_stressed':
    i = 0
  else:
    i = 1
  y_val.append(i)

y_val_label = to_categorical(y_val)


In [ ]:
# loading validation data and converting to an array


x_test = pd.read_csv('/content/drive/MyDrive/Stress_image_detection/data/test.csv')
x_test_img = x_test['filename']

test_images = []

for i in tqdm(x_test_img):
  test_img = image.load_img('/content/drive/MyDrive/Stress_image_detection/Stress_image_dataset/test/' + str(i), target_size=(128, 128, 3), grayscale=False)
  test_img = image.img_to_array(test_img)
  # test_img = tf.image.rgb_to_grayscale(test_img)
  # test_img = np.expand_dims(test_img, axis=0)
  test_img = test_img/255
  test_images.append(test_img)


In [ ]:
X_test = np.array(x_test)

y_test = []

for i in (x_test['class']):
  if i == 'not_stressed':
    i = 0
  else:
    i = 1
  y_test.append(i)

y_test_label = to_categorical(y_test)


In [ ]:
# Training and Evaluating the data

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os


# model = models.Sequential()
# model.add(layers.Conv2D(32, kernel_size=(3,3), activation = 'relu', input_shape = (128, 128, 3)))
# model.add(layers.MaxPooling2D((2,2)))
# model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
# model.add(layers.MaxPooling2D((2,2)))
# model.add(layers.Conv2D(128, (3,3), activation = 'relu'))
# model.add(layers.MaxPooling2D((2,2)))
# model.add(layers.Flatten())
# model.add(layers.Dense(512, activation = 'relu'))
# model.add(layers.Dense(2, activation = 'sigmoid'))

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(1,1,128,128,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

# model.compile(loss = 'binary_crossentropy', optimizer = optimizers.RMSprop(lr = 1e-4), metrics = ['acc'])
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['acc'])


history = model.fit(train_aug.flow(X_train_img, y_train_label, batch_size= 8),
                    steps_per_epoch = len(X_train_img)//8 ,
                    epochs = 50,
                    validation_data = (X_val, y_val_label)
                   )





acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label = 'Trainning acc')
plt.plot(epochs, val_acc, 'b', label = 'Validation acc')
plt.title('Trainning and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label = 'Trainning loss')
plt.plot(epochs, val_acc, 'b', label = 'Validation loss')
plt.title('Trainning and validation loss')
plt.legend()
plt.figure()
plt.show()



In [ ]:
pip install keras-rectified-adam

In [ ]:
pip install keras-adamw

In [ ]:
pip install keras-gradient-accumulation

In [ ]:
pip install keras-adabound

In [ ]:
X_train_new_denoised = np.array(train_denoised)
# initializing the label
# converting the text label to numbers
y_train_denoised = []
for i in x_train['class']:
  if i == 'not_stressed':
    i = 0
  else:
    i = 1
    
  y_train_denoised.append(i)

# converting the label to categorical
y_train_denoised_label = to_categorical(y_train)


In [ ]:
X_val_new_denoised = np.array(val_denoised)
# initializing the label
# converting the text label to numbers


# converting the label to categorical
y_val_denoised_label = to_categorical(y_val)

In [ ]:
X_train_aug = np.add(X_train_img, X_train_new_denoised)
y_train_label_aug = np.add(y_train_label, y_train_denoised_label)

X_val_aug = np.add(X_val, X_val_new_denoised)
y_val_label_aug = np.add(y_val_label, y_val_denoised_label)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os



import sys
from matplotlib import pyplot
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras_adabound import AdaBound
# from keras_radam import RAdam

# Using pre-trained VGG to classify stress
model = VGG16(include_top = False, pooling = 'avg', weights = 'imagenet', input_shape = (128, 128, 3))
for layer in model.layers:
  layer.trainable = True

# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(128, activation = 'relu', kernel_initializer='he_uniform')(flat1)
output = Dense(2, activation= 'softmax')(class1)


# add new model
model = Model(inputs=model.inputs, outputs = output)
# compile model
opt2 = SGD(lr = 0.0001, momentum = 0.9)
opt = AdaBound(lr=0.0001, final_lr=0.001, gamma = 0.0001, weight_decay= 0.)
# opt3 = RAdam(total_steps=10000, warmup_proportion=0.01, min_lr=0.0001)
model.compile(optimizer = opt, loss = 'mse', metrics = ['accuracy'])


# # load model
# model = VGG16(include_top=False, input_shape=(128, 128, 3))
# 	# mark loaded layers as not trainable
# for layer in model.layers:
# 	layer.trainable = False
# 	# add new classifier layers
# flat1 = Flatten()(model.layers[-1].output)
# class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
# output = Dense(1, activation='sigmoid')(class1)
# 	# define new model
# model = Model(inputs=model.inputs, outputs=output)
# 	# compile model
# opt = SGD(lr=0.001, momentum=0.9)
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

Inception_V3

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from sklearn.metrics import accuracy_score
from tensorflow.keras.optimizers import RMSprop
# univariate cnn example
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dense, Convolution1D, MaxPool1D, Flatten, Dropout
from keras.layers import Input
from keras.models import Model
from keras.layers.normalization import BatchNormalization
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix



kfold = KFold(n_splits = 10, shuffle=True)
loo = LeaveOneOut()

# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []
# K-fold Cross Validation model evaluation
fold_no = 1

inputs = np.append(X_train_img, X_val, axis = 0)
targets = np.append(y_train_label, y_val_label, axis = 0)

# for train, test in kfold.split(inputs, targets):
for train, test in loo.split(inputs):
  X_train_img, X_val = inputs[train], inputs[test]
  y_train_label, y_val_label = targets[train], targets[test]
    
  base_model = InceptionV3(input_shape = (128, 128, 3), include_top = False, weights = 'imagenet')
  for layer in base_model.layers:
      layer.trainable = False




  flat1 = Flatten()(base_model.output)
  class1 = Dense(1024, activation='relu')(flat1)
  class1 = Dropout(0.2)(class1)
  output = Dense(2, activation='softmax')(class1)

  inc_model = Model(inputs = base_model.inputs, outputs=output)

  inc_model.compile(optimizer = opt, loss = 'mse', metrics = ['accuracy'])

  callbacks = [EarlyStopping(monitor='val_loss', patience=8)]
              
  # history = inc_model.fit(X_train_img, y_train_label, steps_per_epoch = len(X_train_img)/12, callbacks = callbacks,
  #                     validation_data = (X_val, y_val_label), validation_steps = len(X_val)/12, epochs = 50)
  
  history = inc_model.fit(X_train_img, y_train_label, steps_per_epoch = len(X_train_img)/12, callbacks = callbacks, epochs = 50)

#   scores = inc_model.evaluate(X_val, y_val_label, verbose = 1)

#   print(f'Score for fold {fold_no}: {inc_model.metrics_names[0]} of {scores[0]}; {inc_model.metrics_names[1]} of {scores[1]*100}%')



  
#   acc_per_fold.append(scores[1] * 100)
#   loss_per_fold.append(scores[0])

#   # Increase fold number
#   fold_no = fold_no + 1


# # == Provide average scores ==
# print('------------------------------------------------------------------------')
# print('Score per fold')
# for i in range(0, len(acc_per_fold)):
#   print('------------------------------------------------------------------------')
#   print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
# print('------------------------------------------------------------------------')
# print('Average scores for all folds:')
# print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
# print(f'> Loss: {np.mean(loss_per_fold)}')
# print('------------------------------------------------------------------------')




Resnet

In [ ]:
from tensorflow.keras.applications import ResNet50


kfold = KFold(n_splits = 10, shuffle=True)
loo = LeaveOneOut()
# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []
# K-fold Cross Validation model evaluation
fold_no = 1

inputs = np.append(X_train_img, X_val, axis = 0)
targets = np.append(y_train_label, y_val_label, axis = 0)

# for train, test in kfold.split(inputs, targets):
for train, test in loo.split(inputs):
  X_train_img, X_val = inputs[train], inputs[test]
  y_train_label, y_val_label = targets[train], targets[test]


  res_model = ResNet50(input_shape=(128, 128,3), include_top=False, weights="imagenet")
  for layer in res_model.layers:
      layer.trainable = True


  res_model = Sequential()
  res_model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
  res_model.add(Dense(2, activation='softmax'))

  res_model.compile(optimizer = opt, loss = 'mse', metrics = ['accuracy'])

# callbacks = [EarlyStopping(monitor='val_loss', patience=8),
#              ModelCheckpoint(filepath='resnet_model.h5', monitor='val_loss', save_best_only=True)]
  callbacks = [EarlyStopping(monitor='val_loss', patience=8)]
             
  # history = res_model.fit(X_train_img, y_train_label, steps_per_epoch = len(X_train_img)/12, callbacks = callbacks,
  #                     validation_data = (X_val, y_val_label), validation_steps = len(X_val)/12, epochs = 50)
  
  history = res_model.fit(X_train_img, y_train_label, steps_per_epoch = len(X_train_img)/12, callbacks = callbacks, epochs = 50)
  
#   scores = model.evaluate(X_val, y_val_label, verbose = 1)

#   print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')



  
#   acc_per_fold.append(scores[1] * 100)
#   loss_per_fold.append(scores[0])

#   # Increase fold number
#   fold_no = fold_no + 1


# # == Provide average scores ==
# print('------------------------------------------------------------------------')
# print('Score per fold')
# for i in range(0, len(acc_per_fold)):
#   print('------------------------------------------------------------------------')
#   print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
# print('------------------------------------------------------------------------')
# print('Average scores for all folds:')
# print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
# print(f'> Loss: {np.mean(loss_per_fold)}')
# print('------------------------------------------------------------------------')


VGG19

In [ ]:
import os

import random
import warnings
from sklearn.metrics import confusion_matrix
warnings.filterwarnings('ignore')

#File Operation libraries
import glob
from pathlib import Path

#Visualisation Libraries
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.plotting import plot_confusion_matrix

#Keras
from keras.layers import Dense, Conv2D, MaxPooling2D, AvgPool2D, Input, Dropout, Flatten, BatchNormalization
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import VGG19

from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
import numpy as np
from sklearn.model_selection import LeaveOneOut


kfold = KFold(n_splits = 10, shuffle=True)
loo = LeaveOneOut()
# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []
# K-fold Cross Validation model evaluation
fold_no = 1

inputs = np.append(X_train_img, X_val, axis = 0)
targets = np.append(y_train_label, y_val_label, axis = 0)

# for train, test in kfold.split(inputs, targets):
for train, test in loo.split(inputs):
  X_train_img, X_val = inputs[train], inputs[test]
  y_train_label, y_val_label = targets[train], targets[test]

  vgg19_model = VGG19(weights='imagenet', pooling = 'avg', include_top=False, input_shape = (128, 128, 3))
  for layer in vgg19_model.layers:
    layer.trainable = True


  flat1 = Flatten()(vgg19_model.layers[-1].output)
  class1 = Dense(128, activation = 'relu', kernel_initializer='he_uniform')(flat1)
  output = Dense(2, activation= 'softmax')(class1)

  model = Model(inputs=vgg19_model.inputs, outputs = output)

  model.compile(optimizer = opt, loss = 'mse', metrics = ['accuracy'])

  callbacks = [EarlyStopping(monitor='val_loss', patience=8)]
              
  # history = model.fit(X_train_img, y_train_label, steps_per_epoch = len(X_train_img)/12, callbacks = callbacks,
  #                     validation_data = (X_val, y_val_label), validation_steps = len(X_val)/12, epochs = 50)
  history = model.fit(X_train_img, y_train_label, steps_per_epoch = len(X_train_img)/12, callbacks = callbacks, epochs = 50)
  
  

#   scores = model.evaluate(X_val, y_val_label, verbose = 1)

#   print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')



  
#   acc_per_fold.append(scores[1] * 100)
#   loss_per_fold.append(scores[0])

#   # Increase fold number
#   fold_no = fold_no + 1


# # == Provide average scores ==
# print('------------------------------------------------------------------------')
# print('Score per fold')
# for i in range(0, len(acc_per_fold)):
#   print('------------------------------------------------------------------------')
#   print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
# print('------------------------------------------------------------------------')
# print('Average scores for all folds:')
# print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
# print(f'> Loss: {np.mean(loss_per_fold)}')
# print('------------------------------------------------------------------------')

Xception

In [ ]:
# plotting the model curves
plt.figure(figsize=(10, 10))
acc = history5.history['accuracy']
val_acc = history5.history['val_accuracy']
loss = history5.history['loss']
val_loss = history5.history['val_loss']


pyplot.subplot(211)
pyplot.title('MSE Loss (Adadelta)')
pyplot.plot(loss, color='blue', label='train loss')
pyplot.plot(val_loss, color='orange', label='val loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.show
	# plot accuracy
pyplot.subplot(212)
pyplot.title('Classification Accuracy using Adadelta')
pyplot.plot(acc, color='blue', label='train acc')
pyplot.plot(val_acc, color='orange', label='val acc')

plt.xlabel('epoch')
plt.ylabel('Acc')
plt.legend()
plt.show
	# save plot to file
# filename = sys.argv[0].split('/')[-1]
# pyplot.savefig(filename + '_plot.png')
# pyplot.close()

In [ ]:
plt.figure(figsize=(10, 10))
pyplot.subplot(211)
pyplot.title('MSE Loss of all the optimizers')
plt.plot(history.history['val_loss'],color='blue', label='Adabound')
plt.plot(history2.history['val_loss'],color='green', label='SGD')
plt.plot(history3.history['val_loss'],color='orange', label='Adam')
plt.plot(history4.history['val_loss'],color='Red', label='Rdam')
plt.plot(history5.history['val_loss'],color='purple', label='Adadelta')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show


pyplot.subplot(212)
pyplot.title('Accuracy using of all the optimizers')
plt.plot(history.history['val_accuracy'],color='blue', label='Adabound')
plt.plot(history2.history['val_accuracy'],color='green', label='SGD')
plt.plot(history3.history['val_accuracy'],color='orange', label='Adam')
plt.plot(history4.history['val_accuracy'],color='Red', label='Rdam')
plt.plot(history5.history['val_accuracy'],color='purple', label='Adadelta')
plt.xlabel('Epoch')
plt.ylabel('Acc')
plt.legend()
plt.show



In [ ]:
from sklearn.metrics import accuracy_score
import os

import random
import warnings
from sklearn.metrics import confusion_matrix
warnings.filterwarnings('ignore')

#File Operation libraries
import glob
from pathlib import Path

#Visualisation Libraries
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.plotting import plot_confusion_matrix

#Keras
from keras.layers import Dense, Conv2D, MaxPooling2D, AvgPool2D, Input, Dropout, Flatten, BatchNormalization
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import VGG19

from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
import numpy as np
from sklearn.model_selection import LeaveOneOut

# running the model and getting results

# datagen = ImageDataGenerator(feature_center = True)
# # specify mean values for centering
# datagen.mean = [123.68, 116.779, 103.939]
# # prepare iterator
# train_it = datagen.flow((X_train_img, y_train_label),
#                         class_mode = 'binary',
#                         batch_size = 30, target_size = (128, 128))
# test_it = datagen.flow((X_val, y_val_label),
#                        class_mode = 'binary',
#                        batch_size = 30,
#                        target_size = (128, 128))
# 

# Adabound
# history = model.fit(X_train_img, y_train_label, steps_per_epoch = len(X_train_img),
#                     validation_data = (X_val, y_val_label), validation_steps = len(X_val), epochs = 50)

# SGD

kfold = KFold(n_splits = 10, shuffle=True)
loo = LeaveOneOut()
# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []
# K-fold Cross Validation model evaluation
fold_no = 1

inputs = np.append(X_train_img, X_val, axis = 0)
targets = np.append(y_train_label, y_val_label, axis = 0)

# for train, test in kfold.split(inputs, targets):
for train, test in loo.split(inputs):
  X_train_img, X_val = inputs[train], inputs[test]
  y_train_label, y_val_label = targets[train], targets[test]

  callbacks = [EarlyStopping(monitor='val_loss', patience=8)]
              
  # history = model.fit(X_train_img, y_train_label, steps_per_epoch = len(X_train_img)/12, callbacks = callbacks,
  #                     validation_data = (X_val, y_val_label), validation_steps = len(X_val)/12, epochs = 50)
  
  history = model.fit(X_train_img, y_train_label, steps_per_epoch = len(X_train_img)/12, callbacks = callbacks, epochs = 50)


#   scores = model.evaluate(X_val, y_val_label, verbose = 1)

#   print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')



  
#   acc_per_fold.append(scores[1] * 100)
#   loss_per_fold.append(scores[0])

#   # Increase fold number
#   # fold_no = fold_no + 1


# # == Provide average scores ==
# print('------------------------------------------------------------------------')
# print('Score per fold')
# for i in range(0, len(acc_per_fold)):
#   print('------------------------------------------------------------------------')
#   print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
# print('------------------------------------------------------------------------')
# print('Average scores for all folds:')
# print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
# print(f'> Loss: {np.mean(loss_per_fold)}')
# print('------------------------------------------------------------------------')

# # preds = model.predict(X_test)
# pred_class = np.argmax(preds, axis = 1)

# acc = model.accuracy_score(y_test_label, pred_class, verbose = 0)
# print("VGG16 Model Accuracy without Fine-Tuning: {:.2f}%".format(acc * 100))
# # learning curves
# summarize_diagnostics(history) 



# datagen = ImageDataGenerator(featurewise_center=True)
# 	# specify imagenet mean values for centering
# 	datagen.mean = [123.68, 116.779, 103.939]
# 	# prepare iterator
# 	train_it = datagen.flow_from_directory('dataset_dogs_vs_cats/train/',
# 		class_mode='binary', batch_size=64, target_size=(224, 224))
# 	test_it = datagen.flow_from_directory('dataset_dogs_vs_cats/test/',
# 		class_mode='binary', batch_size=64, target_size=(224, 224))
# 	# fit model
# 	history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
# 		validation_data=test_it, validation_steps=len(test_it), epochs=10, verbose=1)
# 	# evaluate model
# 	_, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
# 	print('> %.3f' % (acc * 100.0))
# 	# learning curves
# 	summarize_diagnostics(history)

Genetic Algortihm

In [ ]:
class CNN(Sequential):
    def __init__(self,nfilters,sfilters):
        super().__init__()
        tensorflow.random.set_seed(0)
        self.vgg = VGG16vgg = VGG16(include_top = False, pooling = 'avg', weights = 'imagenet', input_shape = (128, 128, 3))
        for layer in model.layers:
          layer.trainable = False 
        self.layer_name = 'block1_pool'
        self.my_model = Model(inputs= self.vgg.inputs, outputs = self.vgg.get_layer(self.layer_name).output)
        self.add(Conv2D(nfilters[0],kernel_size=(sfilters[0],sfilters[0]),padding='same',activation='relu'))
        self.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        self.add(Conv2D(nfilters[1],kernel_size=(sfilters[1],sfilters[1]),padding='same',activation='relu'))
        self.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        self.add(Conv2D(nfilters[2],kernel_size=(sfilters[2],sfilters[2]),padding='same',activation='relu'))
        self.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        self.add(Flatten())
        self.add(Dropout(0.3))
        self.add(Dense(512,activation='relu'))
        self.add(Dense(2,activation='softmax'))
        self.opt = AdaBound(lr=0.0001, final_lr=0.01, gamma = 0.0001, weight_decay= 0.)
        self.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
class Genetic:
    
    def __init__(self,pop_size,nlayers,max_nfilters,max_sfilters):
        self.pop_size = pop_size
        self.nlayers = nlayers
        self.max_nfilters = max_nfilters
        self.max_sfilters = max_sfilters
        self.max_acc = 0
        self.best_arch = np.zeros((1,6))
        self.gen_acc = []
    
    def generate_population(self):
        np.random.seed(0)
        pop_nlayers = np.random.randint(1,self.max_nfilters,(self.pop_size,self.nlayers))
        pop_sfilters = np.random.randint(1,self.max_sfilters,(self.pop_size,self.nlayers))
        pop_total = np.concatenate((pop_nlayers,pop_sfilters),axis=1)
        return pop_total
    
    def select_parents(self,pop,nparents,fitness):
        parents = np.zeros((nparents,pop.shape[1]))
        for i in range(nparents):
            best = np.argmax(fitness)
            parents[i] = pop[best]
            fitness[best] = -99999
        return parents
    
    def crossover(self,parents):
        nchild = self.pop_size - parents.shape[0]
        nparents = parents.shape[0]
        child = np.zeros((nchild,parents.shape[1]))
        for i in range(nchild):
            first = i % nparents
            second = (i+1) % nparents
            child[i,:2] = parents[first][:2]
            child[i,2] = parents[second][2]
            child[i,3:5] = parents[first][3:5]
            child[i,5] = parents[second][5]
        return child

    def mutation(self,child):
        for i in range(child.shape[0]):
            val = np.random.randint(1,6)
            ind = np.random.randint(1,4) - 1
            if child[i][ind] + val > 100:
                child[i][ind] -= val
            else:
                child[i][ind] += val
            val = np.random.randint(1,4)
            ind = np.random.randint(4,7) - 1
            if child[i][ind] + val > 20:
                child[i][ind] -= val
            else:
                child[i][ind] += val
        return child
    
    def fitness(self,pop,X,Y,epochs):
        pop_acc = []
        for i in range(pop.shape[0]):
            nfilters = pop[i][0:3]
            sfilters = pop[i][3:]
            model = CNN(nfilters,sfilters)
            H = model.fit(X,Y,batch_size=32,epochs=epochs)
            acc = H.history['accuracy']
            pop_acc.append(max(acc)*100)
        if max(pop_acc) > self.max_acc:
            self.max_acc = max(pop_acc)
            self.best_arch = pop[np.argmax(pop_acc)]
        self.gen_acc.append(max(pop_acc))
        return pop_acc
    
    def smooth_curve(self,factor,gen):
        smoothed_points = []
        for point in self.gen_acc:
            if smoothed_points:
                prev = smoothed_points[-1]
                smoothed_points.append(prev*factor + point * (1-factor))
            else:
                smoothed_points.append(point)
        plt.plot(range(gen+1),smoothed_points,'g',label='Smoothed training acc')
        plt.xticks(np.arange(gen+1))
        plt.legend()
        plt.title('Fitness Accuracy vs Generations')
        plt.xlabel('Generations')
        plt.ylabel('Fitness (%)')
        plt.show()

In [ ]:
pop_size = 10
nlayers = 3
max_nfilters = 100
max_sfilters = 20
epochs = 20
num_generations = 3

genCNN = Genetic(pop_size,nlayers,max_nfilters,max_sfilters)
pop = genCNN.generate_population()

for i in range(num_generations+1):
    pop_acc = genCNN.fitness(pop,X_train_img,y_train_label,epochs)
    print('Best Accuracy at the generation {}: {}'.format(i,genCNN.max_acc))
    parents = genCNN.select_parents(pop,5,pop_acc.copy())
    child = genCNN.crossover(parents)
    child = genCNN.mutation(child)
    pop = np.concatenate((parents,child),axis=0).astype('int')

In [ ]:
genCNN.gen_acc

# Autoencoder

In [ ]:
import numpy as np

import matplotlib.pyplot as plt 

from keras.layers import Conv2D, Input, Dense, Dropout, MaxPool2D, UpSampling2D
from keras.models import Model
from keras.datasets import mnist, cifar10

%matplotlib inline

In [ ]:
# Convolutional Autoencoder

from keras.layers import Conv2DTranspose, BatchNormalization, add, LeakyReLU
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras_adabound import AdaBound

In [ ]:
# Adding noise mean = 0, std = 0.3
noise = 0.3
train_noise = X_train_img + noise * np.random.normal(0, 0.3, size= X_train_img.shape) 
val_noise = X_val + noise * np.random.normal(0, 0.3, size= X_val.shape)

train_noise = np.clip(train_noise, 0, 1)
val_noise = np.clip(val_noise, 0, 1)

In [ ]:
rows = 2 # defining no. of rows in figure
cols = 8 # defining no. of colums in figure

f = plt.figure(figsize=(2*cols,2*rows*2)) # defining a figure 

for i in range(rows):
    for j in range(cols): 
        f.add_subplot(rows*2,cols, (2*i*cols)+(j+1)) # adding sub plot to figure on each iteration
        plt.imshow(train_noise[i*cols + j]) 
        plt.axis("off")
        
    for j in range(cols): 
        f.add_subplot(rows*2,cols,((2*i+1)*cols)+(j+1)) # adding sub plot to figure on each iteration
        plt.imshow(X_train_img[i*cols + j]) 
        plt.axis("off")
        
f.suptitle("Sample Training Data",fontsize=18)
plt.savefig("Cifar-trian.png")

plt.show()

In [ ]:
# Encoder
inputs = Input(shape = (128, 128, 3))

x = Conv2D(128, 3, activation='relu', padding='same')(inputs)
x = BatchNormalization()(x)
x = MaxPool2D()(x)
x = Dropout(0.5)(x)
skip = Conv2D(128, 3, padding='same')(x) # skip connection for decoder
x = LeakyReLU()(skip)
x = BatchNormalization()(x)
x = MaxPool2D()(x)
x = Dropout(0.5)(x)
x = Conv2D(256, 3, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
encoded = MaxPool2D()(x)

# Decoder
x = Conv2DTranspose(256, 3,activation='relu',strides=(2,2), padding='same')(encoded)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Conv2DTranspose(128, 3, activation='relu',strides=(2,2), padding='same')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Conv2DTranspose(128, 3, padding='same')(x)
x = add([x,skip]) # adding skip connection
x = LeakyReLU()(x)
x = BatchNormalization()(x)
decoded = Conv2DTranspose(3, 3, activation='sigmoid',strides=(2,2), padding='same')(x)

autoencoder = Model(inputs, decoded)
opt = AdaBound(lr=0.001, final_lr=0.1, gamma = 0.0001, weight_decay= 0.)
# opt = SGD(lr = 0.0001, momentum = 0.1)
autoencoder.compile(optimizer=opt, loss='categorical_crossentropy')

In [ ]:
# Training
epochs = 50
batch_size = 256

history = autoencoder.fit(train_noise,
                X_train_img,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(val_noise, X_val)
               )


# Defining Figure
f = plt.figure(figsize=(10,7))
f.add_subplot()

#Adding Subplot
plt.plot(history.epoch, history.history['loss'], label = "loss") # Loss curve for training set
plt.plot(history.epoch, history.history['val_loss'], label = "val_loss") # Loss curve for validation set

plt.title("Loss Curve",fontsize=18)
plt.xlabel("Epochs",fontsize=15)
plt.ylabel("Loss",fontsize=15)
plt.grid(alpha=0.3)
plt.legend()
plt.savefig("Loss_curve_stress.png")
plt.show()

In [ ]:
train_denoised = autoencoder.predict(X_train_img)
val_denoised =  autoencoder.predict(X_val)

In [ ]:
# Visualize test images with their denoised images

rows = 4 # defining no. of rows in figure
cols = 12 # defining no. of colums in figure
cell_size = 1.5
f = plt.figure(figsize=(cell_size*cols,cell_size*rows*2)) # defining a figure 
f.tight_layout()
for i in range(rows):
    for j in range(cols): 
        f.add_subplot(rows*2,cols, (2*i*cols)+(j+1)) # adding sub plot to figure on each iteration
        plt.imshow(X_train_vae_new[i*cols + j]) 
        plt.axis("off")
        
    for j in range(cols): 
        f.add_subplot(rows*2,cols,((2*i+1)*cols)+(j+1)) # adding sub plot to figure on each iteration
        plt.imshow(X_val_vae_new[i*cols + j]) 
        plt.axis("off")

f.suptitle("Autoencoder Results - Adabound (lr = 0.001, final lr = 0.1)",fontsize=18)
plt.savefig("Adabound Autoencoder.png")

plt.show()

Autoencoder balance

In [ ]:
from keras.layers import Input, Dense, Lambda, Reshape
from keras.models import Model, Sequential
from keras import regularizers
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn import preprocessing 
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import seaborn as sns
sns.set(style="whitegrid")
np.random.seed(203)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as kr
%matplotlib inline

In [ ]:
# Dataset parameters.
num_features = 49152 # data features (img shape: 28*28).

# Training parameters.
batch_size = 128
epochs = 50
filters = 128

# Network Parameters
hidden_1 = 128 # 1st layer num features.
hidden_2 = 64 # 2nd layer num features (the latent dim).  



# X_train and X_val reshaped for VAE
X_train_vae = np.concatenate([X_train_img, X_val])
X_train_vae = X_train_vae.astype('float32')/255






# # Reshape train and val
X_train_reshape = X_train_img.reshape([-1, 128, 128, 3])
X_val_reshape = X_val.reshape([-1, 128, 128, 3])

X_train_reshape = X_train_reshape.astype(np.float32, copy=False)
X_val_reshape = X_val_reshape.astype(np.float32, copy=False)


# x_train_re = X_train_img.astype('float32') / 255.
# x_val_re = X_val.astype('float32') / 255.
# x_train_re = X_train_img.reshape((len(X_train_img), np.prod(X_train_img.shape[1:])))
# x_val_re = X_val.reshape((len(X_val), np.prod(X_val.shape[1:])))


In [ ]:
# plot predictions

def plot_predictions(y_true, y_pred):    
    f, ax = plt.subplots(2, 10, figsize=(15, 4))
    for i in range(10):
        ax[0][i].imshow(np.reshape(y_true[i], (128, 128, 3)), aspect='auto')
        ax[1][i].imshow(np.reshape(y_pred[i], (128, 128, 3)), aspect='auto')
    plt.tight_layout()

# plot digits



def plot_digits(X, y, encoder, batch_size=128):
    """Plots labels and MNIST digits as function of 2D latent vector

    Parameters:
    ----------
    encoder: Model
        A Keras Model instance
    X: np.ndarray
        Test data
    y: np.ndarray
        Test data labels
    batch_size: int
        Prediction batch size
    """
    # display a 2D plot of the digit classes in the latent space
    z_mean, _, _ = encoder.predict(X, batch_size=batch_size)
    plt.figure(figsize=(12, 10))
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c=y)
    plt.colorbar()
    plt.xlabel("z[0] Latent Dimension")
    plt.ylabel("z[1] Latent Dimension")
    plt.show()
    

# Plot manfold

def generate_manifold(decoder):
    """Generates a manifold of MNIST digits from a random noisy data.

    Parameters:
    ----------
    decoder: Model
        A Keras Model instance
    """
    
    # display a 30x30 2D manifold of digits
    n = 30
    digit_size = 28
    figure = np.zeros((digit_size * n, digit_size * n))
    
    # linearly spaced coordinates corresponding to the 2D plot
    # of digit classes in the latent space
    grid_x = np.linspace(-4, 4, n)
    grid_y = np.linspace(-4, 4, n)[::-1]
    
    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi]])
            x_decoded = decoder.predict(z_sample)
            digit = x_decoded[0].reshape(digit_size, digit_size)
            figure[i * digit_size: (i + 1) * digit_size,
                   j * digit_size: (j + 1) * digit_size] = digit        
    
    plt.figure(figsize=(10, 10))
    start_range = digit_size // 2
    end_range = n * digit_size + start_range + 1
    pixel_range = np.arange(start_range, end_range, digit_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("z[0] Latent Dimension")
    plt.ylabel("z[1] Latent Dimension")
    plt.imshow(figure, cmap='Greys_r')
    plt.show()

In [ ]:
# Variaional Encoder

inputs = kr.Input(shape=(num_features, ))
encoder = kr.layers.Dense(hidden_1, activation='sigmoid')(inputs)
encoder = kr.layers.Dense(hidden_2, activation='sigmoid')(encoder)
encoder_model = kr.Model(inputs, encoder, name='encoder')
encoder_model.summary()

In [ ]:
# Decoder

latent_dim = kr.Input(shape=(hidden_2, ))
decoder = kr.layers.Dense(hidden_1, activation='sigmoid')(latent_dim)
decoder = kr.layers.Dense(num_features, activation='sigmoid')(decoder)
decoder_model = kr.Model(latent_dim, decoder, name='decoder')


In [ ]:
# Autoencoder model

outputs = decoder_model(encoder_model(inputs))
stress_model = kr.Model(inputs, outputs )
stress_model.compile(optimizer='adam', loss='mse')


In [ ]:
# trainingg
stress_model.fit(x=X_train_reshape, y=X_train_reshape, batch_size=batch_size, shuffle=False, epochs=epochs)

In [ ]:

y_pred = stress_model.predict(X_val_reshape)
plot_predictions(X_val, y_pred)

In [ ]:
from keras import backend as K

class Sampling(Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [ ]:
# new autoencoder

original_dim = 128 * 128 * 3
input_shape = (128, 128, 3)
hidden_dim = 512

# The bigger this is, more accurate the network is but 2 is for illustration purposes.
latent_dim = 10
 

# encoder

encoder_inputs = Input(shape=(128, 128, 3))
x = Conv2D(128, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = Conv2D(256, 3, activation="relu", strides=2, padding="same")(x)
x = Flatten()(x)
x = Dense(64, activation="relu")(x)
z_mean = Dense(latent_dim, name="z_mean")(x)
z_log_var = Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

# implement the decoder

latent_inputs = Input(shape=(latent_dim,))
x = Dense(32 * 32 * 256, activation="relu")(latent_inputs)
x = Reshape((32, 32, 256))(x)
x = Conv2DTranspose(256, 3, activation="relu", strides=2, padding="same")(x)
x = Conv2DTranspose(128, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = Conv2DTranspose(3, 3, activation="sigmoid", padding="same")(x)
decoder = Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()


# VAE implementation
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }



vae = VAE(encoder, decoder)
vae.compile(Adam())
vae.fit(X_train_vae, epochs=50, batch_size=128)


# vae = Model(inputs, outputs, name='vae')
# # reconstruction loss
# reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
# reconstruction_loss *= (original_dim)


# kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)

# kl_loss = K.sum(kl_loss, axis=-1)
# kl_loss *= -0.5
# vae_loss = K.mean(reconstruction_loss + kl_loss)
# vae.add_loss(vae_loss)


# vae.add_loss(vae_loss)

vae.compile(optimizer='adam')
# # print(outputs.shape)

# vae.fit(X_train_reshape, X_train_reshape, epochs=epochs, batch_size=batch_size, validation_data=(X_val_reshape, X_val_reshape))

# inputs = kr.layers.Input(shape=(num_features, ), name='input')
# x = kr.layers.Dense(hidden_dim, activation='relu')(inputs)
# z_mean = kr.layers.Dense(latent_dim, name='z_mean')(x)
# z_log_var = kr.layers.Dense(latent_dim, name='z_log_var')(x)

# # reparamaterization
# z = kr.layers.Lambda(sampling, name='z')([z_mean, z_log_var])
# # instantiate encoder model
# encoder = kr.Model(inputs, [z_mean, z_log_var, z], name='encoder')


# # Decoder
# latent_inputs = kr.layers.Input(shape=(latent_dim,), name='z_sampling')
# x = kr.layers.Dense(hidden_dim, activation='relu')(latent_inputs)
# outputs = kr.layers.Dense(num_features, activation='sigmoid')(x)

# # instantiate decoder model
# decoder = kr.Model(latent_inputs, outputs, name='decoder')

# # # VAE model = encoder + decoder
# outputs = decoder(encoder(inputs)[2])  # Select the Z value from outputs of the encoder
# vae = kr.Model(inputs, outputs, name='vae')


# # Reconstruction loss
# reconstruction_loss = tf.losses.mean_squared_error(inputs, outputs)
# reconstruction_loss = reconstruction_loss * num_features

# # KL Divergence loss
# kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
# kl_loss = -0.5 * tf.reduce_sum(kl_loss, axis=-1)
# vae_loss = tf.reduce_mean(reconstruction_loss + kl_loss)

# vae.add_loss(vae_loss)
# vae.compile(optimizer='adam')

# vae.fit(X_train_reshape, epochs=epochs, batch_size=batch_size, validation_data=(X_val_reshape, None))

In [ ]:
encoder.summary()

In [ ]:
import matplotlib.pyplot as plt


def plot_latent_space(vae, n=30, figsize=15):
    # display a n*n 2D manifold of digits
    digit_size = 128
    scale = 1.0
    figure = np.zeros((digit_size * n, digit_size * n))
    # linearly spaced coordinates corresponding to the 2D plot
    # of digit classes in the latent space
    grid_x = np.linspace(-scale, scale, n)
    grid_y = np.linspace(-scale, scale, n)[::-1]

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi]])
            x_decoded = vae.decoder.predict(z_sample)
            digit = x_decoded[0].reshape(digit_size, digit_size, 3)
            figure[
                i * digit_size : (i + 1) * digit_size, 
                j * digit_size : (j + 1) * digit_size, 
            ] = digit

    plt.figure(figsize=(figsize, figsize))
    start_range = digit_size // 2
    end_range = n * digit_size + start_range
    pixel_range = np.arange(start_range, end_range, digit_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.imshow(figure, cmap="Greys_r")
    plt.show()


plot_latent_space(vae)

In [ ]:
def plot_label_clusters(vae, data, labels):
    # display a 2D plot of the digit classes in the latent space
    z_mean, _, _ = vae.encoder.predict(data)
    plt.figure(figsize=(12, 10))
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c=labels)
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.show()


X_train_re = X_train_img.reshape([-1, 128, 128, 3])
y_train_re = np.array(y_train)

plot_label_clusters(vae, X_train_re, y_train_re)

In [ ]:
plot_digits(X_val_reshape, y_val, encoder)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sys

import keras
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, MaxPool2D, Flatten, BatchNormalization
from keras.layers import Conv1D, MaxPool1D, CuDNNLSTM, Reshape, Conv1DTranspose
from keras.layers import Input, Dense, Dropout, Activation, Add, Concatenate
from keras.datasets import cifar10
from keras import regularizers
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.optimizers import SGD, Adam, RMSprop, Adadelta
import keras.backend as K
from keras.objectives import mean_squared_error
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils

from sklearn.utils import class_weight
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer, RobustScaler, StandardScaler

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

Variational Autoencodder

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.layers import *
from keras.models import *
from keras.losses import *
from keras.callbacks import *
from keras.optimizers import *
from keras import backend as K
import matplotlib.pyplot as plt

In [ ]:
# helper functions

# reparameterization trick
# instead of sampling from Q(z|X), sample eps = N(0,I)
# z = z_mean + sqrt(var)*eps
def sampling(args):
    z_mean, z_log_var = args
    batch = tf.keras.backend.shape(z_mean)[0]
    dim = tf.keras.backend.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.keras.backend.exp(0.5 * z_log_var) * epsilon

def plot_results(models, data, batch_size=128, model_name="vae_mnist"):
    """Plots labels and MNIST digits as function of 2-dim latent vector
    # Arguments:
        models (tuple): encoder and decoder models
        data (tuple): test data and label
        batch_size (int): prediction batch size
        model_name (string): which model is using this function
    """

    encoder, decoder = models
    x_test, y_test = data
    os.makedirs(model_name, exist_ok=True)

    filename = os.path.join(model_name, "vae_mean.png")
    # display a 2D plot of the digit classes in the latent space
    z_mean, _, _ = encoder.predict(x_test,
                                   batch_size=batch_size)
    plt.figure(figsize=(12, 10))
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c=y_test)
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.savefig(filename)
    plt.show()

    filename = os.path.join(model_name, "digits_over_latent.png")
    # display a 30x30 2D manifold of digits
    n = 30
    digit_size = 28
    figure = np.zeros((digit_size * n, digit_size * n))
    # linearly spaced coordinates corresponding to the 2D plot
    # of digit classes in the latent space
    grid_x = np.linspace(-4, 4, n)
    grid_y = np.linspace(-4, 4, n)[::-1]

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi]])
            x_decoded = decoder.predict(z_sample)
            digit = x_decoded[0].reshape(digit_size, digit_size)
            figure[i * digit_size: (i + 1) * digit_size,
                   j * digit_size: (j + 1) * digit_size] = digit

    plt.figure(figsize=(10, 10))
    start_range = digit_size // 2
    end_range = n * digit_size + start_range + 1
    pixel_range = np.arange(start_range, end_range, digit_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.imshow(figure, cmap='Greys_r')
    plt.savefig(filename)
    plt.show()

In [ ]:
# Reshape parameter to apply zmean

X_train_va = np.reshape(X_train_img, [-1, 49152])
X_val_va = np.reshape(X_val, [-1, 49152])
X_train_va = X_train_va.astype('float32') / 255
X_val_va = X_val_va.astype('float32') / 255

# y_pred = stress_model.predict(X_val_reshape)

In [ ]:
original_dim = 49152
# network parameters
input_shape = (49152, )
intermediate_dim = 512
batch_size = 128
latent_dim = 5
epochs = 50

# VAE model = encoder + decoder
# build encoder model
inputs = tf.keras.layers.Input(shape=input_shape, name='encoder_input')
x = tf.keras.layers.Dense(intermediate_dim, activation='relu')(inputs)
z_mean = tf.keras.layers.Dense(latent_dim, name='z_mean')(x)
z_log_var = tf.keras.layers.Dense(latent_dim, name='z_log_var')(x)

# use reparameterization trick to push the sampling out as input
z = tf.keras.layers.Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# instantiate encoder model
encoder = tf.keras.models.Model(inputs, [z_mean, z_log_var, z], name='encoder')
# encoder.summary()
# tf.keras.utils.plot_model(encoder, to_file='vae_mlp_encoder.png', show_shapes=True)

# build decoder model
latent_inputs = tf.keras.layers.Input(shape=(latent_dim,), name='z_sampling')
x = tf.keras.layers.Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = tf.keras.layers.Dense(original_dim, activation='softmax')(x)

# instantiate decoder model
decoder = tf.keras.models.Model(latent_inputs, outputs, name='decoder')
# decoder.summary()
# tf.keras.utils.plot_model(decoder, to_file='vae_mlp_decoder.png', show_shapes=True)

# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = tf.keras.models.Model(inputs, outputs, name='vae_mlp')

models = (encoder, decoder)
# data = (x_test, y_test)
# reconstruction_loss = tf.keras.losses.mse(inputs, outputs)
reconstruction_loss = tf.keras.losses.binary_crossentropy(inputs, outputs)

reconstruction_loss *= original_dim

kl_loss = 1 + z_log_var - tf.keras.backend.square(z_mean) - tf.keras.backend.exp(z_log_var)
kl_loss = tf.keras.backend.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = tf.keras.backend.mean(reconstruction_loss + kl_loss)
# vae.add_loss(vae_loss)
vae.compile(optimizer='adam', loss='categorical_crossentropy')

In [ ]:
vae.fit(X_train_va, epochs=epochs, batch_size=batch_size, validation_data=(X_val_va, None))

1D tests using wesad

In [ ]:
from sklearn.preprocessing import StandardScaler 
from pandas import read_csv
from numpy import set_printoptions
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA, SparsePCA
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras import regularizers
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn import preprocessing 
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import seaborn as sns
sns.set(style="whitegrid")
np.random.seed(203)
import scipy.sparse as sparse

In [ ]:
filename = '/content/combined-swell-classification-hrv-test-dataset.csv'

data = read_csv(filename)



not_stressed_sw = data[data['Condition Label'] == 0].sample(6000)

not_stressed_sw_L = data[data['Condition Label'] == 0].sample(500)



stressed_sw =  data[data['Condition Label'] == 1]
stressed2_sw = data[data['Condition Label'] == 2]
stressed_sw = stressed_sw.append(stressed2_sw).sample(6000)

stressed_sw_L = stressed_sw.append(stressed2_sw).sample(500)


# stressed_SSSQ
# y1 = df[:, 65]
# y2 = df[:, 66]
# y1_label = []
# for i in y1:
#   if i == 1:
#     y1_label.append(1)
#   elif i == 2:
#     y1_label.append(1)
#   else:
#     y1_label.append(0)



# for i in y2:
#   if i == 1:
#     y2_label.append(1)
#   elif i == 2:
#     y2_label.append(1)
#   else:
#     y2_label.append(0)


df = not_stressed_sw.append(stressed_sw).sample(frac=1).reset_index(drop=True)

df_sw_L = not_stressed_sw_L.append(stressed_sw_L).sample(frac=1).reset_index(drop=True)

X = df.drop(['subject_id','condition','NasaTLX class','NasaTLX Label','Condition Label'], axis = 1).values

X_sw_L = df_sw_L.drop(['subject_id','condition','NasaTLX class','NasaTLX Label','Condition Label'], axis = 1).values
print(X)

X_t = df.drop(['HR', 'MEAN_RR','RMSSD','condition','NasaTLX class','NasaTLX Label','Condition Label'], axis = 1).values
X_flat = X_t.flatten()
X_flat_pos = abs(X_flat)

y1 = df["Condition Label"].values
y1 = np.where(y1>1, 1, y1)


y1_sw_L = df_sw_L["Condition Label"].values
y1_sw_L = np.where(y1_sw_L>1, 1, y1_sw_L)



print(y1)

In [ ]:
filename2 = '/content/wesad-classification-hrv-test-dataset.csv'

data2 = read_csv(filename2)



not_stressed2 = data2[data2['condition label'] == 0].sample(6000)
not_stressed_L = data2[data2['condition label'] == 0].sample(500)



stressed_2 =  data2[data2['condition label'] == 1]
stressed_2_2 = data2[data2['condition label'] == 2]
stressed_2 = stressed_2.append(stressed_2_2).sample(6000)
stressed_2_L = stressed_2.append(stressed_2_2).sample(500)


# stressed_SSSQ
# y1 = df[:, 65]
# y2 = df[:, 66]
# y1_label = []
# for i in y1:
#   if i == 1:
#     y1_label.append(1)
#   elif i == 2:
#     y1_label.append(1)
#   else:
#     y1_label.append(0)



# for i in y2:
#   if i == 1:
#     y2_label.append(1)
#   elif i == 2:
#     y2_label.append(1)
#   else:
#     y2_label.append(0)


df2 = not_stressed2.append(stressed_2).sample(frac=1).reset_index(drop=True)
df_L = not_stressed_L.append(stressed_2_L).sample(frac = 1).reset_index(drop = True)




X1 = df2.drop(['subject id','condition','SSSQ class','SSSQ Label','condition label'], axis = 1).values
print(X1)

X_L = df_L.drop(['subject id','condition','SSSQ class','SSSQ Label','condition label'], axis = 1).values

X1_flat = X1.flatten()
X1_flat_pos = abs(X1_flat)


y2 = df2["condition label"].values
y_L = df_L['condition label'].values
y_L = np.where(y_L > 1, 1, y_L)


y2 = np.where(y2>1, 1, y2)

df3 = df2.drop(['subject id','condition','SSSQ class','SSSQ Label','condition label'], axis = 1)

# df3['condition label'] = y2.tolist()
print(df3.shape)



Statistical Tests

In [ ]:
# T-test
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
from scipy.stats import pearsonr
from scipy.stats import mannwhitneyu
from scipy.stats import wilcoxon
from scipy.stats import friedmanchisquare
from scipy.stats import spearmanr
from scipy.stats import chi2_contingency
# from scipy.stats import kruska

# stat, p = ttest_ind(X_flat, X1_flat)
# print('Stat = %0.3f, p = %0.3f' % (stat, p))
# if p > 0.05:
#   print('the datasets are not different')
# else:
#   print('The dataset are different')

stat_p, p_p = pearsonr(X_flat, X1_flat)
print('Stat = %0.3f, p = %0.3f' % (stat_p, p_p))
if p_p > 0.05:
  print('The dataset are not different')
else:
  print('The dataset are different')



stat_m, p_m = mannwhitneyu(X_flat, X1_flat)
print('Stat_m = %0.3f, p_m = %0.3f' % (stat_m, p_m))
if p_m > 0.05:
  print('The dataset are not different')
else:
  print('The dataset are different')


stat_w, p_w = wilcoxon(X_flat, X1_flat)
print('Stat_w = %0.3f, p_w = %0.3f' % (stat_w, p_w))
if p_w > 0.05:
  print('The dataset are not different')
else:
  print('The dataset are different')

stat_s, p_s = spearmanr(X_flat, X1_flat)
print('stat_s = %0.3f, p_s = %0.3f' % (stat_s, p_s))
if p_s > 0.05:
  print('The dataset are not different')
else:
  print('The dataset are different')


stat_c, p_c = chi2_contingency(X_flat_pos, X1_flat_pos)
print('Stat_c = %0.3f, p_c = %0.3f' % (stat_c, p_c))
if p_c > 0.05:
  print('The dataset are not different')
else:
  print('The dataset are different ')




# stat_c, p_c = pearsonr(X_flat, X1_flat)
# print('Stat = %0.3f, p = %0.3f' % (stat_c, p_c))
# if p_c > 0.05:
#   print('the datasets are not different')
# else:
#   print('The dataset are different')

# stat_m, p_m = mannwhitneyu(X_flat, X1_flat)
# print('Stat = %0.3f, p = %0.3f' % (stat_m, p_m))
# if p_m > 0.05:
#   print('the datasets are not different')
# else:
#   print('The dataset are different')

# stat_w, p_w = wilcoxon(X_flat, X1_flat)
# print('Stat = %0.3f, p = %0.3f' % (stat_w, p_w))
# if p_w > 0.05:
#   print('the datasets are not different')
# else:
#   print('The dataset are different')

# stat_s, p_s = spearmanr(X_flat, X1_flat)
# print('Stat = %0.3f, p = %0.3f' % (stat_s, p_s))
# if p_s > 0.05:
#   print('the datasets are not different')
# else:
#   print('The dataset are different')


# stat_ch, p_ch = chi2_contingency(X_flat, X1_flat)
# print('Stat = %0.3f, p = %0.3f' % (stat_ch, p_ch))
# if p_ch > 0.05:
#   print('the datasets are not different')
# else:
#   print('The dataset are different')


In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from numpy import std
from numpy import mean

from sklearn.model_selection import LeaveOneOut
cv = LeaveOneOut()

train_x2, test_x2, train_y2, test_y2 = train_test_split(rep_x2, rep_y2, test_size=0.2, stratify=rep_y2,
                     random_state=0)




# The "accuracy" scoring is proportional to the number of correct classifications
RF = RandomForestClassifier(random_state=0) 
RF.fit(train_x2, train_y2)
rf_pred = RF.predict(test_x2)





# GB.fit(train_x2, train_y2)
# gb_pred = GB.predict(test_x2)

# rfecv = RFECV(estimator=clf_rf_4, step=1, cv=5,scoring='accuracy')   #5-fold cross-validation
# rfecv = rfecv.fit(train_xd, train_yd)
# cv = LeaveOneOut()
# scores = cross_val_score(RF, rep_xL, rep_yL, scoring='accuracy', cv=cv, n_jobs=-1)


# print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

# print('Optimal number of features :', rfecv.n_features_)
# print('Best features :', df3.columns[rfecv.support_])

In [ ]:
def bias_variance_decomp(estimator, X_train, y_train, X_test, y_test,
                         loss='0-1_loss', num_rounds=200, random_seed=None,
                         **fit_params):
    """
    estimator : object
        A classifier or regressor object or class implementing both a
        `fit` and `predict` method similar to the scikit-learn API.
    X_train : array-like, shape=(num_examples, num_features)
        A training dataset for drawing the bootstrap samples to carry
        out the bias-variance decomposition.
    y_train : array-like, shape=(num_examples)
        Targets (class labels, continuous values in case of regression)
        associated with the `X_train` examples.
    X_test : array-like, shape=(num_examples, num_features)
        The test dataset for computing the average loss, bias,
        and variance.
    y_test : array-like, shape=(num_examples)
        Targets (class labels, continuous values in case of regression)
        associated with the `X_test` examples.
    loss : str (default='0-1_loss')
        Loss function for performing the bias-variance decomposition.
        Currently allowed values are '0-1_loss' and 'mse'.
    num_rounds : int (default=200)
        Number of bootstrap rounds (sampling from the training set)
        for performing the bias-variance decomposition. Each bootstrap
        sample has the same size as the original training set.
    random_seed : int (default=None)
        Random seed for the bootstrap sampling used for the
        bias-variance decomposition.
    fit_params : additional parameters
        Additional parameters to be passed to the .fit() function of the
        estimator when it is fit to the bootstrap samples.
    Returns
    ----------
    avg_expected_loss, avg_bias, avg_var : returns the average expected
        average bias, and average bias (all floats), where the average
        is computed over the data points in the test set.
    Examples
    -----------
    For usage examples, please see
    http://rasbt.github.io/mlxtend/user_guide/evaluate/bias_variance_decomp/
    """
    supported = ['0-1_loss', 'mse']
    if loss not in supported:
        raise NotImplementedError('loss must be one of the following: %s' %
                                  supported)

    for ary in (X_train, y_train, X_test, y_test):
        if hasattr(ary, 'loc'):
            raise ValueError('The bias_variance_decomp does not '
                             'support pandas DataFrames yet. '
                             'Please check the inputs to '
                             'X_train, y_train, X_test, y_test. '
                             'If e.g., X_train is a pandas '
                             'DataFrame, try passing it as NumPy array via '
                             'X_train=X_train.values.')

    rng = np.random.RandomState(random_seed)

    if loss == '0-1_loss':
        dtype = np.int64
    elif loss == 'mse':
        dtype = np.float64

    all_pred = np.zeros((num_rounds, y_test.shape[0]), dtype=dtype)

    for i in range(num_rounds):
        X_boot, y_boot = _draw_bootstrap_sample(rng, X_train, y_train)

        # Keras support
        if estimator.__class__.__name__ in ['Sequential', 'Functional']:

            # reset model
            for ix, layer in enumerate(estimator.layers):
                if hasattr(estimator.layers[ix], 'kernel_initializer') and \
                        hasattr(estimator.layers[ix], 'bias_initializer'):
                    weight_initializer = \
                        estimator.layers[ix].kernel_initializer
                    bias_initializer = estimator.layers[ix].bias_initializer

                    old_weights, old_biases = \
                        estimator.layers[ix].get_weights()

                    estimator.layers[ix].set_weights([
                        weight_initializer(shape=old_weights.shape),
                        bias_initializer(shape=len(old_biases))])

            estimator.fit(X_boot, y_boot, **fit_params)
            pred = estimator.predict(X_test).reshape(1, -1)
        else:
            pred = estimator.fit(
                X_boot, y_boot, **fit_params).predict(X_test)
        all_pred[i] = pred

    if loss == '0-1_loss':
        main_predictions = np.apply_along_axis(lambda x:
                                               np.argmax(np.bincount(x)),
                                               axis=0,
                                               arr=all_pred)

        avg_expected_loss = np.apply_along_axis(lambda x:
                                                (x != y_test).mean(),
                                                axis=1,
                                                arr=all_pred).mean()

        avg_bias = np.sum(main_predictions != y_test) / y_test.size

        var = np.zeros(pred.shape)

        for pred in all_pred:
            var += (pred != main_predictions).astype(np.int)
        var /= num_rounds

        avg_var = var.sum()/y_test.shape[0]

    else:
        avg_expected_loss = np.apply_along_axis(
            lambda x:
            ((x - y_test)**2).mean(),
            axis=1,
            arr=all_pred).mean()

        main_predictions = np.mean(all_pred, axis=0)

        avg_bias = np.sum((main_predictions - y_test)**2) / y_test.size
        avg_var = np.sum((main_predictions - all_pred)**2) / all_pred.size

    return avg_expected_loss, avg_bias, avg_var

In [ ]:
from mlxtend.evaluate import bias_variance_decomp

mse, bias, var = bias_variance_decomp(RF, train_x2, train_y2, test_x2, test_y2, loss='mse', num_rounds=50, random_seed=None)
# summarize results
print('MSE: %.3f' % mse)
print('Bias: %.3f' % bias)
print('Variance: %.3f' % var)

In [ ]:
from mlxtend.evaluate import bias_variance_decomp

# Bias and Variances
variance = np.var(rf_pred)
print('Variance = %0.3f' % (variance))



variance_gb = np.var(gb_pred)
print(variance_gb)

SSE_gb  = np.mean((np.mean(gb_pred) - rep_yL)**2)
gb_bias = SSE_gb - variance_gb


SSE = np.mean((np.mean(rf_pred) - rep_yL)**2)
rf_bias = SSE - variance




print('Rf Bias: %0.3f' % (rf_bias))

print('GB bias: %0.3f' % (gb_bias))



In [ ]:
%pip install mlxtend --upgrade

In [ ]:
# Statistical test
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from numpy import std
from numpy import mean
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import log
from sklearn.ensemble import GradientBoostingClassifier





train_xd, test_xd, train_yd, test_yd = train_test_split(X1, y2, test_size=0.2, stratify=y2,
                     random_state=0)

# The "accuracy" scoring is proportional to the number of correct classifications
RF = RandomForestClassifier() 
RF.fit(train_xd, train_yd)
num_params = 63
yhat = RF.predict(test_xd)
mse = mean_squared_error(test_yd, yhat)
print('MSE: %.3f' % mse)


# GB
GB = GradientBoostingClassifier(n_estimators=100, 
                                learning_rate=1.0, 
                                max_depth=1, 
                                random_state=0)

GB = GB.fit(train_xd, train_yd)
pred_y = GB.predict(test_xd)
mse_gb = mean_squared_error(test_yd, pred_y)
print(mse_gb)



# lower AIC and BIC means less information loss and overfitting
def calculate_aic(n, mse, num_params):
	aic = n * log(mse) + 2 * num_params
	return aic


aic = calculate_aic(len(train_yd), mse, num_params)

aic_gb = calculate_aic(len(train_yd), mse_gb, num_params)

print('AIC: %.3f' % aic)
print('AIC for GB: %.3f' % aic_gb)

def calculate_bic(n, mse, num_params):
  bic = n * log(mse) + num_params * log(n)
  return bic

bic = calculate_bic(len(train_yd), mse, num_params)
print('BIC: %.3f' % bic)

bic_gb = calculate_bic(len(train_yd), mse_gb, num_params)
print('BIC of GB: %.3f' % bic_gb)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score of number of selected features")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [ ]:
clf_rf_5 = RandomForestClassifier()      
clr_rf_5 = clf_rf_5.fit(train_xd,train_yd)
importances = clr_rf_5.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf_rf_5.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(train_xd.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest

plt.figure(1, figsize=(14, 13))
plt.title("Feature importances")
plt.bar(range(train_xd.shape[1]), importances[indices],
       color="g", yerr=std[indices], align="center")
plt.xticks(range(train_xd.shape[1]), df3.columns[indices],rotation=90)
plt.xlim([-1, train_xd.shape[1]])
plt.show()

In [ ]:
# Train Data

classes={0:"Not Stressed", 1:"Stressed"}
plt.figure(figsize=(10,2))
for i in range(0,2):
    plt.subplot(1,2,i + 1)
    all_samples_indexes = np.where(y1 == i)[0]
    rand_samples_indexes = np.random.randint(0, len(all_samples_indexes), 3)
    rand_samples = X[rand_samples_indexes]
    plt.plot(rand_samples.transpose())
    plt.title("Samples of class " + classes[i], loc="left", fontdict={'fontsize':8})

In [ ]:
# Test Data
# Train Data

classes={0:"Not Stressed", 1:"Stressed"}
plt.figure(figsize=(10,2))
for i in range(0,2):
    plt.subplot(1,2,i + 1)
    all_samples_indexes = np.where(y2 == i)[0]
    rand_samples_indexes = np.random.randint(0, len(all_samples_indexes), 3)
    rand_samples = X1[rand_samples_indexes]
    plt.plot(rand_samples.transpose())
    plt.title("Samples of class " + classes[i], loc="right", fontdict={'fontsize':8})

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


corr = df2.corr()

plt.figure(figsize= (100, 100))
sns.heatmap(corr, annot = True)

In [ ]:
# X_sup = df.loc[:,['MEAN_RR','MEDIAN_RR','HR','MEAN_RR_LOG','MEAN_RR_SQRT','HR_SQRT']]
# print(X_sup)


In [ ]:
# X_sup2 = df2.loc[:,['MEAN_RR','MEDIAN_RR','HR','MEAN_RR_LOG','MEAN_RR_SQRT','HR_SQRT']].values

# print(X_sup2)


# X_sup2 = df2.iloc[:,[0,1,6,14,26,32,33,52,54,56]]
X_sup2 = df2.iloc[:,[0,1,6,14,26,32]]
print(X_sup2)

In [ ]:
def tsne_plot(x1, y1, name="graph.png"):
    tsne = TSNE(n_components=2, random_state=0)
    X_t = tsne.fit_transform(x1)

    plt.figure(figsize=(12, 8))
    plt.scatter(X_t[np.where(y1 == 0), 0], X_t[np.where(y1 == 0), 1], marker='o', color='g', linewidth='1', alpha=0.8, label='Not Stressed')
    plt.scatter(X_t[np.where(y1 == 1), 0], X_t[np.where(y1 == 1), 1], marker='o', color='r', linewidth='1', alpha=0.8, label='Stressed')

    plt.legend(loc='best');
    plt.savefig(name);
    plt.show();
    
tsne_plot(X, y1, "original.png")

In [ ]:
def tsne_plot(x1, y1, name="graph.png"):
    tsne = TSNE(n_components=2, random_state=0)
    X_t = tsne.fit_transform(x1)

    plt.figure(figsize=(12, 8))
    plt.scatter(X_t[np.where(y1 == 0), 0], X_t[np.where(y1 == 0), 1], marker='o', color='g', linewidth='1', alpha=0.8, label='Not Stressed')
    plt.scatter(X_t[np.where(y1 == 1), 0], X_t[np.where(y1 == 1), 1], marker='o', color='r', linewidth='1', alpha=0.8, label='Stressed')

    plt.legend(loc='best');
    plt.savefig(name);
    plt.show();
    
tsne_plot(X1, y2, "original.png")

In [ ]:
print(X)

In [ ]:
# PCA
pca = KernelPCA(n_components=4)
fit = pca.fit(X_sup)
X_sup_new = pca.transform(X_sup)
print(X_sup_new.shape)

In [ ]:
pca2 = KernelPCA(n_components=4)
fit = pca2.fit(X_sup2)
X_sup2_new = pca2.transform(X_sup2)
print(X_sup2_new.shape)

In [ ]:
sparsepca = make_pipeline(StandardScaler(),
                          SparsePCA(n_components=9,
                                    alpha=0.0001,
                                    random_state=0,
                                    n_jobs=-1))
fit = sparsepca.fit(X_sup2)
X_sup2_new = sparsepca.transform(X_sup2)


In [ ]:
def plot_hist(class_number,size,min_, bins):
    img=df3.loc[df3['condition label']==class_number].values
    img=img[:,min_:size]
    img_flatten=img.flatten()
    print(img.shape)
    final1=np.arange(min_,size)
    print(final1.shape)
    for i in range (img.shape[0]-1):
        tempo1=np.arange(min_,size)
        final1=np.concatenate((final1, tempo1), axis=None)
    print(len(final1))
    print(len(img_flatten))
    plt.hist2d(final1, img_flatten, bins=(bins),cmap=plt.cm.jet)
    plt.show()

In [ ]:
plot_hist(0, 10,0, 10)


In [ ]:
plot_hist(1, 10, 0, 10)

Scatter Plot using various dimensional reduction

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import datasets

from sklearn.decomposition import (PCA, IncrementalPCA,
                                   KernelPCA, TruncatedSVD,
                                   FastICA, MiniBatchDictionaryLearning,
                                   SparsePCA)

from sklearn.manifold import (Isomap,
                              LocallyLinearEmbedding)

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.random_projection import (GaussianRandomProjection,
                                       SparseRandomProjection)

from sklearn.neighbors import (KNeighborsClassifier,
                               NeighborhoodComponentsAnalysis)
                               
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

%matplotlib inline

In [ ]:
# Train, Test split for dimensional reduction

train_xd, test_xd, train_yd, test_yd = train_test_split(X_sup2, y2, test_size=0.2, stratify=y2,
                     random_state=0)

In [ ]:
random_state = 0

pca = make_pipeline(StandardScaler(),
                    PCA(n_components=4,
                        random_state=random_state))

inc_pca = make_pipeline(StandardScaler(),
                        IncrementalPCA(n_components=4))

# kernel : “linear” | “poly” | “rbf” | “sigmoid” | “cosine” | “precomputed”
kpca = make_pipeline(StandardScaler(),
                     KernelPCA(kernel="cosine",
                               n_components=4,
                               gamma=None,
                               fit_inverse_transform=True,
                               random_state=random_state,
                               n_jobs=1))

sparsepca = make_pipeline(StandardScaler(),
                          SparsePCA(n_components=4,
                                    alpha=0.0001,
                                    random_state=random_state,
                                    n_jobs=-1))


SVD = make_pipeline(StandardScaler(),
                    TruncatedSVD(n_components=4,
                                 algorithm='randomized',
                                 random_state=random_state,
                                 n_iter=5))


GRP = make_pipeline(StandardScaler(),
                    GaussianRandomProjection(n_components=4,
                                             eps = 0.5,
                                             random_state=random_state))

# lda = make_pipeline(StandardScaler(),
#                     LinearDiscriminantAnalysis(n_components=4))

nca = make_pipeline(StandardScaler(),
                    NeighborhoodComponentsAnalysis(n_components=4,
                                                   random_state=random_state))


SRP = make_pipeline(StandardScaler(),
                    SparseRandomProjection(n_components=4,
                                           density = 'auto',
                                           eps = 0.5,
                                           random_state=random_state,
                                           dense_output = False))


isomap = make_pipeline(StandardScaler(),
                       Isomap(n_components=4,
                              n_jobs = 4,
                              n_neighbors = 5))


miniBatchDictLearning = make_pipeline(StandardScaler(),
                                      MiniBatchDictionaryLearning(n_components=4,
                                                                  batch_size = 200,
                                                                  alpha = 1,
                                                                  n_iter = 25,
                                                                  random_state=random_state))


# FastICA = make_pipeline(StandardScaler(),
#                         FastICA(n_components=3,
#                                 algorithm = 'parallel',
#                                 whiten = True,
#                                 max_iter = 100,
#                                 random_state=random_state))


lle = make_pipeline(StandardScaler(),
                    LocallyLinearEmbedding(n_components=4,
                                           n_neighbors = 10,
                                           method = 'modified',
                                           n_jobs = 4,
                                           random_state=random_state))



In [ ]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import GradientBoostingClassifier
# train_x, val_x, train_y, val_y = train_test_split(_x, rep_y, test_size=0.25)
# GB = GradientBoostingClassifier(n_estimators=100, 
#                                 learning_rate=1.0, 
#                                 max_depth=1, 
#                                 random_state=0)


# Make a list of the methods to be compared
dim_reduction_methods = {'PCA': pca, 
                        #  'LDA': lda, 
                         'NCA': nca, 
                         'INC PCA': inc_pca, 
                         'KPCA':kpca, 
                         'Sparced PCA': sparsepca, 
                         'SVD': SVD, 
                         'GRP' : GRP, 
                         'SRP': SRP, 
                         'IsoMap': isomap, 
                         'MBD': miniBatchDictLearning, 
                        #  'ICA': FastICA, 
                         'LLE': lle}


plt.figure(figsize=(24, 36))

for j,(name, model) in enumerate(dim_reduction_methods.items()):
    plt.subplot(5, 3, j + 1, aspect='auto')

    # Fit the method's model
    model.fit(train_xd, train_yd)

    # Fit a nearest neighbor classifier on the embedded training set
    clf_rf_4.fit(model.transform(train_xd), train_yd)

    # Compute the nearest neighbor accuracy on the embedded test set
    # acc_GB = GB.score(model.transform(test_xd), test_yd)
    acc_RF = clf_rf_4.score(model.transform(test_xd), test_yd)

    # Fit the methons using the fitted model
    X_embedded = model.transform(X_sup2)
    print(X_embedded.shape)
    
    # Creating a dataframe to easily plot the sample label
    df = pd.DataFrame(np.concatenate((X_embedded, np.reshape(y2, (-1, 1))), axis=1))

    # Plot the projected points and show the evaluation score
    plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=y2, s=20, cmap='Set1')
    plt.title("{}, RF Test accuracy = {:.2f}".format(name, acc_RF))
    plt.colorbar()
    
    # Label the data distributions
    for i, number in enumerate(test_yd):
        plt.annotate(number,
                     df.loc[df[2]==number,[0,1]].mean(),
                     horizontalalignment='center',
                     verticalalignment='center',
                     weight='bold',
                     size='20')
    
plt.show()

In [ ]:
## input layer 
input_layer = Input(shape=(X.shape[1],))

## encoding part
encoded = Dense(100, activation='tanh', activity_regularizer=regularizers.l1(10e-4))(input_layer)
encoded = Dense(50, activation='relu')(encoded)
# encoded = Dense(25, activation='relu')(encoded)


## decoding part
# encoded = Dense(25, activation='relu')(encoded)
decoded = Dense(50, activation='tanh')(encoded)
decoded = Dense(100, activation='tanh')(decoded)

## output layer
output_layer = Dense(X.shape[1], activation='relu')(decoded)

In [ ]:
## input layer 
input_layer2 = Input(shape=(X_sup2.shape[1],))

## encoding part
encoded2 = Dense(100, activation='tanh', activity_regularizer=regularizers.l1(10e-4))(input_layer2)
encoded2 = Dense(50, activation='relu')(encoded2)
# encoded2 = Dense(25, activation='relu')(encoded2)


## decoding part
# encoded2 = Dense(25, activation='relu')(encoded2)
decoded2 = Dense(50, activation='tanh')(encoded2)
decoded2 = Dense(100, activation='tanh')(decoded2)

## output layer
output_layer2 = Dense(X_sup2.shape[1], activation='relu')(decoded2)

In [ ]:
# VGG
## input layer 
input_layer2 = Input(shape=(X1.shape[1],))

## encoding part
encoded2 = Dense(100, activation='tanh', activity_regularizer=regularizers.l1(10e-4))(input_layer2)
encoded2 = Dense(50, activation='relu')(encoded2)
# encoded2 = Dense(25, activation='relu')(encoded2)


## decoding part
# encoded2 = Dense(25, activation='relu')(encoded2)
decoded2 = Dense(50, activation='tanh')(encoded2)
decoded2 = Dense(100, activation='tanh')(decoded2)

## output layer
output_layer2 = Dense(X1.shape[1], activation='relu')(decoded2)

In [ ]:
# LOOCV
## input layer 
input_layer_L = Input(shape=(X_L.shape[1],))

## encoding part
encoded2 = Dense(100, activation='tanh', activity_regularizer=regularizers.l1(10e-4))(input_layer_L)
encoded2 = Dense(50, activation='relu')(encoded2)
# encoded2 = Dense(25, activation='relu')(encoded2)


## decoding part
# encoded2 = Dense(25, activation='relu')(encoded2)
decoded2 = Dense(50, activation='tanh')(encoded2)
decoded2 = Dense(100, activation='tanh')(decoded2)

## output layer
output_layer_L = Dense(X_L.shape[1], activation='relu')(decoded2)

In [ ]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from keras_adabound import AdaBound
from keras_radam import RAdam

autoencoder = Model(input_layer, output_layer)
# opt = SGD(lr = 0.01, momentum = 0.9)
opt = AdaBound(lr=0.01, final_lr=0.1, gamma = 0.01, weight_decay= 0.)
# opt = RAdam(total_steps=10000, warmup_proportion=0.1, min_lr=0.1)
autoencoder.compile(optimizer=opt, loss="mse")

In [ ]:
# LOOCV autoencoder
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from keras_adabound import AdaBound
from keras_radam import RAdam

autoencoder_L = Model(input_layer_L, output_layer_L)
# opt = SGD(lr = 0.01, momentum = 0.9)
opt = AdaBound(lr=0.01, final_lr=0.1, gamma = 0.01, weight_decay= 0.)
# opt = RAdam(total_steps=10000, warmup_proportion=0.1, min_lr=0.1)
autoencoder_L.compile(optimizer=opt, loss="mse")

In [ ]:
pip install keras-adabound

In [ ]:
pip install keras-rectified-adam

In [ ]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from keras_adabound import AdaBound
from keras_radam import RAdam

autoencoder2 = Model(input_layer2, output_layer2)
# opt = SGD(lr = 0.01, momentum = 0.9)
opt = AdaBound(lr=0.01, final_lr=0.1, gamma = 0.01, weight_decay= 0.)
opt2 = RAdam(total_steps=10000, warmup_proportion=0.01, min_lr=0.001)
opt3 = Adam(lr = 0.001)
opt4 = SGD(lr = 0.001, momentum = 0.9)
opt5 = 'adadelta'
# opt = RAdam(total_steps=10000, warmup_proportion=0.1, min_lr=0.01)
autoencoder2.compile(optimizer=opt, loss="mse")

In [ ]:
import time

# Make a list of the methods to be compared
optimizers = {'RAdam': opt2, 
                         'Adam': opt3, 
                         'SGD':opt4, 
                         'Adadelta': opt5, 
                         'AdaBound': opt, 
                        #  'LDA
                         }


plt.figure(figsize=(24, 36))
plt.title("Visualizing loss with various optimizers (StandardScaler)")
fig, ax = plt.subplots(figsize=(5,5))
for j,(name, opt) in enumerate(optimizers.items()):
    # plt.subplot(5, 3, j + 1, aspect='auto')
    

    # Fit the method's model
    autoencoder2.compile(optimizer = opt, loss = 'mse')
    # x_scale2 = StandardScaler().fit_transform(X1)
    # x_norm2, x_stress2 = x_scale2[y2 == 0], x_scale2[y2 == 1]

    history = autoencoder2.fit(x_norm2[0:12000], x_norm2[0:12000], 
                batch_size = 256, epochs = 50, 
                shuffle = True, validation_split = 0.20);

    
    # plt.plot(history.history['loss'])

    plt.plot(history.history['val_loss'], label = "val_loss of " + str(name))
    plt.legend()
    plt.xlabel('epoch')
    plt.ylabel('loss')

    time.sleep(3)
plt.show()
    
    
    
#     model.fit(train_xd, train_yd)

#     # Fit a nearest neighbor classifier on the embedded training set
#     clf_rf_4.fit(model.transform(train_xd), train_yd)

#     # Compute the nearest neighbor accuracy on the embedded test set
#     # acc_GB = GB.score(model.transform(test_xd), test_yd)
#     acc_RF = clf_rf_4.score(model.transform(test_xd), test_yd)

#     # Fit the methons using the fitted model
#     X_embedded = model.transform(X_sup2)
#     print(X_embedded.shape)
    
#     # Creating a dataframe to easily plot the sample label
#     df = pd.DataFrame(np.concatenate((X_embedded, np.reshape(y2, (-1, 1))), axis=1))

#     # Plot the projected points and show the evaluation score
#     plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=y2, s=20, cmap='Set1')
#     plt.title("{}, RF Test accuracy = {:.2f}".format(name, acc_RF))
#     plt.colorbar()
    
#     # Label the data distributions
#     for i, number in enumerate(test_yd):
#         plt.annotate(number,
#                      df.loc[df[2]==number,[0,1]].mean(),
#                      horizontalalignment='center',
#                      verticalalignment='center',
#                      weight='bold',
#                      size='20')
    
# plt.show()

In [ ]:
x_scale = MinMaxScaler().fit_transform(X)
x_norm, x_stress = x_scale[y1 == 0], x_scale[y1 == 1]

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.utils.vis_utils import plot_model

# x_scale2 = MinMaxScaler().fit_transform(X_sup2)
x_scale2 = MinMaxScaler().fit_transform(X1)
# x_scale2 = StandardScaler().fit_transform(X1)
x_norm2, x_stress2 = x_scale2[y2 == 0], x_scale2[y2 == 1]



In [ ]:
# LOOCV autoencoder

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.utils.vis_utils import plot_model

# x_scale2 = MinMaxScaler().fit_transform(X_sup2)
x_scale_L = MinMaxScaler().fit_transform(X_L)
# x_scale2 = StandardScaler().fit_transform(X1)
x_norm_L, x_stress_L = x_scale_L[y_L == 0], x_scale_L[y_L == 1]

In [ ]:
autoencoder.fit(x_norm[0:12000], x_norm[0:12000], 
                batch_size = 256, epochs = 50, 
                shuffle = True, validation_split = 0.20);
autoencoder.summary()

In [ ]:
# autoencoder2.fit(x_norm2[0:6000], x_norm2[0:6000], 
#                 batch_size = 256, epochs = 50, 
#                 shuffle = True, validation_split = 0.20);
# autoencoder2.summary()

autoencoder2.fit(x_norm2[0:12000], x_norm2[0:12000], 
                batch_size = 256, epochs = 50, 
                shuffle = True, validation_split = 0.20);
autoencoder2.summary()
# plot_model(autoencoder2, to_file='autoencoder_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
# LOOCV autoencoder

autoencoder_L.fit(x_norm_L[0:1000], x_norm_L[0:1000], 
                batch_size = 128, epochs = 50, 
                shuffle = True, validation_split = 0.20);


In [ ]:
plt.figure(figsize=(5, 5))
plt.title("Visualizing loss with various Adabound")
# fig, ax = plt.subplots(figsize=(5,5))

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'], label = "val_loss") 
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')

  
plt.show()

In [ ]:
hidden_representation = Sequential()
hidden_representation.add(autoencoder.layers[0])
hidden_representation.add(autoencoder.layers[1])
hidden_representation.add(autoencoder.layers[2])
hidden_representation.add(autoencoder.layers[3])
# hidden_representation.add(autoencoder.layers[4])
# hidden_representation.add(autoencoder.layers[5])

In [ ]:
hidden_representation2 = Sequential()
hidden_representation2.add(autoencoder2.layers[0])
hidden_representation2.add(autoencoder2.layers[1])
hidden_representation2.add(autoencoder2.layers[2])
hidden_representation2.add(autoencoder2.layers[3])
# hidden_representation2.add(autoencoder2.layers[4])
# hidden_representation2.add(autoencoder2.layers[5])

In [ ]:
# LOOCV autoencoder

hidden_representation_L = Sequential()
hidden_representation_L.add(autoencoder_L.layers[0])
hidden_representation_L.add(autoencoder_L.layers[1])
hidden_representation_L.add(autoencoder_L.layers[2])
hidden_representation_L.add(autoencoder_L.layers[3])

In [ ]:
norm_hid_rep = hidden_representation.predict(x_norm[:12000])
stress_hid_rep = hidden_representation.predict(x_stress)

In [ ]:
# norm_hid_rep2 = hidden_representation2.predict(x_norm2[:6000])
# stress_hid_rep2 = hidden_representation2.predict(x_stress2)

norm_hid_rep2 = hidden_representation2.predict(x_norm2[:12000])
stress_hid_rep2 = hidden_representation2.predict(x_stress2)

In [ ]:
# LOOCV autoencoder
# norm_hid_rep2 = hidden_representation2.predict(x_norm2[:6000])
# stress_hid_rep2 = hidden_representation2.predict(x_stress2)

norm_hid_rep_L = hidden_representation_L.predict(x_norm_L[:1000])
stress_hid_rep_L = hidden_representation_L.predict(x_stress_L)

In [ ]:
rep_x = np.append(norm_hid_rep, stress_hid_rep, axis = 0)
y_norm = np.zeros(norm_hid_rep.shape[0])
y_stress = np.ones(stress_hid_rep.shape[0])
rep_y = np.append(y_norm, y_stress)
tsne_plot(rep_x, rep_y, "latent_representation.png")

In [ ]:
rep_x2 = np.append(norm_hid_rep2, stress_hid_rep2, axis = 0)
y_norm2 = np.zeros(norm_hid_rep2.shape[0])
y_stress2 = np.ones(stress_hid_rep2.shape[0])
rep_y2 = np.append(y_norm2, y_stress2)
tsne_plot(rep_x2, rep_y2, "latent_representation.png")

In [ ]:
# LOOOCV autoencoder data

rep_xL = np.append(norm_hid_rep_L, stress_hid_rep_L, axis = 0)
y_norm_L = np.zeros(norm_hid_rep_L.shape[0])
y_stress_L = np.ones(stress_hid_rep_L.shape[0])
rep_yL = np.append(y_norm_L, y_stress_L)
tsne_plot(rep_xL, rep_yL, "latent_representation.png")


In [ ]:
# LR classification

train_x, val_x, train_y, val_y = train_test_split(rep_x, rep_y, test_size=0.25)
clf = LogisticRegression(solver="lbfgs").fit(train_x, train_y)
pred_y = clf.predict(val_x)

print ("Accuracy Score: ", accuracy_score(val_y, pred_y))

In [ ]:
from sklearn.model_selection import validation_curve
from sklearn.linear_model import Ridge
from yellowbrick.model_selection import LearningCurve

train_x2, val_x2, train_y2, val_y2 = train_test_split(rep_x2, rep_y2, test_size=0.25)
RF.fit(train_x2, train_y2)
pred_yd = RF.predict(val_x2)

print ("Accuracy Score: ", accuracy_score(val_y2, pred_yd))


precision = precision_score(val_y2, pred_yd)
print('Precision: %f' % precision)


recall = recall_score(val_y2, pred_yd)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(val_y2, pred_yd)
print('F1 score: %f' % f1)
 
# kappa
kappa = cohen_kappa_score(val_y2, pred_yd)
print('Cohens kappa: %f' % kappa)
# ROC AUC
auc = roc_auc_score(val_y2, pred_yd)
print('ROC AUC: %f' % auc)


In [ ]:
train_x2, val_x2, train_y2, val_y2 = train_test_split(rep_x2, rep_y2, test_size=0.25)
clf2 = LogisticRegression(solver="lbfgs").fit(train_x2, train_y2)
pred_y2 = clf2.predict(val_x2)

print ("Accuracy Score: ", accuracy_score(val_y2, pred_y2))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
train_x, val_x, train_y, val_y = train_test_split(rep_x, rep_y, test_size=0.25)
GB = GradientBoostingClassifier(n_estimators=100, 
                                learning_rate=1.0, 
                                max_depth=1, 
                                random_state=0)

GB_clf = GB.fit(train_x, train_y)
pred_y = GB_clf.predict(val_x)

print ("Accuracy Score: ", accuracy_score(val_y, pred_y))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score

train_x2, val_x2, train_y2, val_y2 = train_test_split(rep_x2, rep_y2, test_size=0.25)
GB = GradientBoostingClassifier(n_estimators=100, 
                                learning_rate=1.0, 
                                max_depth=1, 
                                random_state=0)

GB_clf2 = GB.fit(train_x2, train_y2)
pred_y2 = GB_clf2.predict(val_x2)
hat_y2 = GB_clf2.predict(val_x2)

print ("Accuracy Score: ", accuracy_score(val_y2, pred_y2))

precision = precision_score(val_y2, hat_y2)
print('Precision: %f' % precision)


recall = recall_score(val_y2, hat_y2)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(val_y2, hat_y2)
print('F1 score: %f' % f1)
 
# kappa
kappa = cohen_kappa_score(val_y2, hat_y2)
print('Cohens kappa: %f' % kappa)
# ROC AUC
auc = roc_auc_score(val_y2, hat_y2)
print('ROC AUC: %f' % auc)


In [ ]:
# NN classification

from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(12, input_dim=50, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_x, train_y, epochs=50, batch_size=10)
_, accuracy = model.evaluate(val_x, val_y)

print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
# opt2 = SGD(lr = 0.0001, momentum = 0.9)
# opt2 = AdaBound(lr=0.0001, final_lr=0.001, gamma = 0.01, weight_decay= 0.)
opt2 = RAdam(total_steps=10000, warmup_proportion=0.1, min_lr=0.001)

In [ ]:
# univariate cnn example
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dense, Convolution1D, MaxPool1D, Flatten, Dropout
from keras.layers import Input
from keras.models import Model
from keras.layers.normalization import BatchNormalization
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix

def network(X_train,y_train,X_test,y_test):
    

    im_shape=(X_train.shape[1],1)
    inputs_cnn=Input(shape=(im_shape), name='inputs_cnn')
    conv1_1=Convolution1D(64, (6), activation='relu', input_shape=im_shape)(inputs_cnn)
    conv1_1=BatchNormalization()(conv1_1)
    pool1=MaxPool1D(pool_size=(3), strides=(2), padding="same")(conv1_1)
    conv2_1=Convolution1D(64, (6), activation='relu', input_shape=im_shape)(pool1)
    conv2_1=BatchNormalization()(conv2_1)
    pool2=MaxPool1D(pool_size=(2), strides=(2), padding="same")(conv2_1)
    conv3_1=Convolution1D(64, (3), activation='relu', input_shape=im_shape)(pool2)
    conv3_1=BatchNormalization()(conv3_1)
    pool3=MaxPool1D(pool_size=(2), strides=(2), padding="same")(conv3_1)
    flatten=Flatten()(pool3)
    dense_end1 = Dense(64, activation='relu')(flatten)
    dense_end2 = Dense(32, activation='relu')(dense_end1)
    main_output = Dense(1, activation='sigmoid', name='main_output')(dense_end2)
    
    
    model = Model(inputs= inputs_cnn, outputs=main_output)
    model.compile(optimizer='adam', loss='mse',metrics = ['accuracy'])
    
    
    callbacks = [EarlyStopping(monitor='val_loss', patience=8),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

    history=model.fit(X_train, y_train,epochs=40,callbacks=callbacks, batch_size=32,validation_data=(X_test,y_test))
    model.load_weights('best_model.h5')
    return(model,history)


In [ ]:
# Using RFE features only
# univariate cnn example
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dense, Convolution1D, MaxPool1D, Flatten, Dropout
from keras.layers import Input
from keras.models import Model
from keras.layers.normalization import BatchNormalization
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint

def network(X_train,y_train,X_test,y_test):
    

    im_shape=(X_train.shape[1],1)
    inputs_cnn=Input(shape=(im_shape), name='inputs_cnn')
    conv1_1=Convolution1D(64, (1), activation='relu', input_shape=im_shape)(inputs_cnn)
    conv1_1=BatchNormalization()(conv1_1)
    pool1=MaxPool1D(pool_size=(3), strides=(2), padding="same")(conv1_1)
    conv2_1=Convolution1D(64, (1), activation='relu', input_shape=im_shape)(pool1)
    conv2_1=BatchNormalization()(conv2_1)
    pool2=MaxPool1D(pool_size=(2), strides=(2), padding="same")(conv2_1)
    conv3_1=Convolution1D(64, (1), activation='relu', input_shape=im_shape)(pool2)
    conv3_1=BatchNormalization()(conv3_1)
    pool3=MaxPool1D(pool_size=(2), strides=(2), padding="same")(conv3_1)
    flatten=Flatten()(pool3)
    dense_end1 = Dense(64, activation='relu')(flatten)
    dense_end2 = Dense(32, activation='relu')(dense_end1)
    main_output = Dense(1, activation='sigmoid', name='main_output')(dense_end2)
    
    
    model = Model(inputs= inputs_cnn, outputs=main_output)
    model.compile(optimizer='adam', loss='mse',metrics = ['accuracy'])
    
    
    callbacks = [EarlyStopping(monitor='val_loss', patience=8),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

    history=model.fit(X_train, y_train,epochs=40,callbacks=callbacks, batch_size=32,validation_data=(X_test,y_test))
    model.load_weights('best_model.h5')
    return(model,history)

1D VGG 16 model

In [ ]:
from keras.optimizers import Adam
from keras.initializers import RandomUniform

# train_x, val_x, train_y, val_y = train_test_split(rep_x2, rep_y2, test_size=0.25)

def make_vgg161D_model(X_train, y_train, X_test, y_test):
  I = (X_train.shape[1], 1)
  inputs_cnn = Input(shape=(I), name='inputs_cnn')
  conv1_1 = Convolution1D(filters=64,kernel_size=(1),padding="same", activation="relu")(inputs_cnn)
  conv1_1 = Convolution1D(filters=64,kernel_size=(1),padding="same", activation="relu")(conv1_1)
  conv1_1=BatchNormalization()(conv1_1)
  pool1 = MaxPool1D(pool_size=(2),strides=(2))(conv1_1)
  conv2_1 = Convolution1D(filters=64, kernel_size=(1), padding="same", activation="relu")(pool1)
  conv2_1 = Convolution1D(filters=64, kernel_size=(1), padding="same", activation="relu")(conv2_1)
  conv2_1=BatchNormalization()(conv2_1)
  pool2 = MaxPool1D(pool_size=(2),strides=(2))(conv2_1)
  conv3_1 = Convolution1D(filters=64, kernel_size=(1), padding="same", activation="relu")(pool2)  
  conv3_1 = Convolution1D(filters=64, kernel_size=(1), padding="same", activation="relu")(conv3_1)
  conv3_1 = Convolution1D(filters=64, kernel_size=(1), padding="same", activation="relu")(conv3_1)
  conv3_1=BatchNormalization()(conv3_1)
  pool3 = MaxPool1D(pool_size=(2),strides=(2))(conv3_1)
  conv4_1 = Convolution1D(filters=64, kernel_size=(1), padding="same", activation="relu")(pool3)
  conv4_1 = Convolution1D(filters=64, kernel_size=(1), padding="same", activation="relu")(conv4_1)
  conv4_1 = Convolution1D(filters=64, kernel_size=(1), padding="same", activation="relu")(conv4_1)
  conv4_1=BatchNormalization()(conv4_1)
  pool4 = MaxPool1D(pool_size=(2),strides=(2))(conv4_1)
  conv5_1 = Convolution1D(filters=64, kernel_size=(1), padding="same", activation="relu")(pool4)
  conv5_1 = Convolution1D(filters=64, kernel_size=(1), padding="same", activation="relu")(conv5_1)
  conv5_1 = Convolution1D(filters=64, kernel_size=(1), padding="same", activation="relu")(conv5_1)
  conv5_1=BatchNormalization()(conv5_1)
  pool5 = MaxPool1D(pool_size=(2),strides=(2))(conv5_1)
  flatten = Flatten()(pool5)
  dense1 = Dense(64, activation = 'relu')(flatten)
  dense2 = Dense(32, activation = 'relu')(dense1)
  main_output = Dense(1, activation = 'sigmoid', name = 'main_output')(dense2)

  model = Model(inputs= inputs_cnn, outputs=main_output)
 
  # lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(0.00001, decay_steps=10000, decay_rate=0.75)
  # adam = Adam(learning_rate=lr_schedule, beta_1=0.9, beta_2=0.999, amsgrad=False)
  opt = Adam(lr=0.0001)
  model.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])

  callbacks = [EarlyStopping(monitor='val_loss', patience=8),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

  history=model.fit(X_train, y_train,epochs=40,callbacks=callbacks, batch_size=32,validation_data=(X_test,y_test))
  model.load_weights('best_model.h5')
  return(model,history)









#   model = Sequential()
# model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
# model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

In [ ]:
def evaluate_model(history,X_test,y_test,model):
    scores = model.evaluate((X_test),y_test, verbose=1)
    print("Accuracy: %.2f%%" % (scores[1]*100))
    
    print(history)
    fig1, ax_acc = plt.subplots()
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Model - Accuracy')
    plt.legend(['Training', 'Validation'], loc='lower right')
    plt.show()
    
    fig2, ax_loss = plt.subplots()
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Model- Loss')
    plt.legend(['Training', 'Validation'], loc='upper right')
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.show()
    target_names=['0','1']
    
    y_true=[]
    for element in y_test:
        y_true.append(np.argmax(element))
    prediction_proba=model.predict(X_test)
    prediction=np.argmax(prediction_proba,axis=1)
    cnf_matrix = confusion_matrix(y_true, prediction)
    print(cnf_matrix)

In [ ]:
model, history = network(train_x, train_y, val_x, val_y)

In [ ]:
# VGG
model_vgg, history_vgg = make_vgg161D_model(train_x, train_y, val_x, val_y)

In [ ]:
# Transfer Learning

D1 = Dense(64,activation="relu")(model.output)
D2 = Dense(32, activation="relu")(D1)
O = Dense(1, activation='sigmoid')(D2)
model_transfer = Model(inputs=model.input, outputs=O)

for layer in model_transfer.layers[:-3]:
    # layer.trainable = False
    layer.trainable = True
for layer in model_transfer.layers[-3:]:
    layer.trainable = True



    
model_transfer.compile(optimizer=Adam(lr= 0.0001), loss='mse', metrics=['accuracy'])
callbacks = [EarlyStopping(monitor='val_loss', patience=8),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

# plot_model(model_transfer, show_shapes=True, to_file='VGG_transfer_learning.png')
history_transfer = model_transfer.fit(train_x2, 
                    train_y2, 
                    validation_split=0.2,
                    epochs=40,
                    callbacks = callbacks,
                    batch_size=32,
                    validation_data=(val_x2,val_y2),
                    verbose=1)

In [ ]:
# VGG transfer learning

# Transfer Learning

D1 = Dense(64,activation="relu")(model_vgg.output)
D2 = Dense(32, activation="relu")(D1)
O = Dense(1, activation='sigmoid')(D2)
modelvgg_transfer = Model(inputs=model_vgg.input, outputs=O)

for layer in modelvgg_transfer.layers[:-3]:
    # layer.trainable = False
    layer.trainable = True
for layer in modelvgg_transfer.layers[-3:]:
    layer.trainable = True



    
modelvgg_transfer.compile(optimizer=Adam(lr= 0.0001), loss='mse', metrics=['accuracy'])
callbacks = [EarlyStopping(monitor='val_loss', patience=8),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

# plot_model(model_transfer, show_shapes=True, to_file='VGG_transfer_learning.png')
historyvgg_transfer = modelvgg_transfer.fit(train_x2, 
                    train_y2, 
                    validation_split=0.2,
                    epochs=40,
                    callbacks = callbacks,
                    batch_size=32,
                    validation_data=(val_x2,val_y2),
                    verbose=1)

In [ ]:
# bias and variance test
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        modelvgg_transfer, train_x2, train_y2, val_x2, val_y2, 
        loss='mse',
        num_rounds=2,
        random_seed=None,
        epochs=40,
        verbose=1) 


print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)


In [ ]:
# bias and variance test
avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        modelvgg_transfer, train_x2, train_y2, val_x2, val_y2, 
        loss='mse',
        num_rounds=4,
        random_seed=None,
        epochs=10,
        verbose=1) 


print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)

In [ ]:
import math
# yhat_classes = model_transfer.predict(val_x2, verbose=0)
# yhat_classes = np.round(yhat_classes[:,0])
# print(yhat_classes)

# precision = precision_score(val_y2, yhat_classes)
# print('Precision: %f' % precision)
# # recall: tp / (tp + fn)
# recall = recall_score(val_y2, yhat_classes)
# print('Recall: %f' % recall)
# # f1: 2 tp / (2 tp + fp + fn)
# f1 = f1_score(val_y2, yhat_classes)
# print('F1 score: %f' % f1)
 
# # kappa
# kappa = cohen_kappa_score(val_y2, yhat_classes)
# print('Cohens kappa: %f' % kappa)
# # ROC AUC
# auc = roc_auc_score(val_y2, yhat_classes)
# print('ROC AUC: %f' % auc)
# # confusion matrix


# AIC and BIC

# Statistical test
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from numpy import std
from numpy import mean
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import log
from sklearn.ensemble import GradientBoostingClassifier


# lower AIC and BIC means less information loss and overfitting
def calculate_aic(n, mse, num_params):
	aic = n * log(mse) + 2 * num_params
	return aic


def calculate_bic(n, mse, num_params):
  bic = n * log(mse) + num_params * log(n)
  return bic


# The "accuracy" scoring is proportional to the number of correct classifications


num_params = 50
yhat = model_transfer.predict(val_x2, verbose = 0)
yhat = np.round(yhat[:,0])
mse_transfer_cnn = mean_squared_error(val_y2, yhat)
print('MSE_CNN: %0.3f' % mse_transfer_cnn)

bic_CNN = calculate_bic(len(train_y2), mse_transfer_cnn, num_params)
print('BIC_CNN: %0.3f' % bic_CNN)

aic_CNN = calculate_aic(len(train_y2), mse_transfer_cnn, num_params)
print('AIC: %0.3f' % aic_CNN)


num_params = 50
yhat_vgg = modelvgg_transfer.predict(val_x2, verbose = 0)
yhat_vgg = np.round(yhat_vgg[:,0])
mse_transfer_vgg = mean_squared_error(val_y2, yhat_vgg)
print('MSE_vgg: %0.3f' % mse_transfer_vgg)

bic_vgg = calculate_bic(len(train_y2), mse_transfer_vgg, num_params)
print('BIC_vgg: %0.3f' % bic_vgg)

aic_vgg = calculate_aic(len(train_y2), mse_transfer_vgg, num_params)
print('AIC_vgg: %0.3f' % aic_vgg)



In [ ]:
def plot_learning(history):
    fig, ax = plt.subplots()
    plt.subplot(211)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Acc using 1D VGG16 algorithm, autoencoder and transfer learning (swell input, wesad output)')
    plt.legend(["accuracy", 'val_accuracy'])
    fig2, ax_loss = plt.subplots()
    plt.subplot(212)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'], label = "val_loss")
    plt.title('Loss using 1D VGG16, autoencoder and transfer learning (swell input, wesad output)')
    plt.legend(["loss", "val_loss"])
    plt.show()

In [ ]:
plot_learning(history_transfer)

In [ ]:
model,history=network(train_x,train_y,val_x,val_y)

In [ ]:
model2,history2=network(train_x2,train_y2,val_x2,val_y2)
# plot_model(model2,  show_shapes=True, to_file='CNN_autoencoder.png')


In [ ]:
evaluate_model(history_transfer,val_x2,val_y2,model_transfer)

In [ ]:
evaluate_model(history2,val_x2,val_y2,model2)


Transfer Learning

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import tensorflow as tf

from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
%config IPCompleter.greedy=True

In [ ]:
from tensorflow.keras.layers import Input, Conv1D, MaxPool1D, Activation, Add, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

In [ ]:
# input_shape = (50, 1)
input_shape = (8,6,1)

def make_model(final_layer_size=1):
    I = Input(input_shape)
    C = Conv1D(filters=32, kernel_size=2)(I)

    C11 = Conv1D(filters=32, kernel_size=2, activation='relu', padding='same')(C)
    C12 = Conv1D(filters=32, kernel_size=2, padding='same')(C11)
    A11 = Add()([C, C12])
    R11 = Activation(activation='relu')(A11)
    M11 = MaxPool1D(pool_size=2, strides=2)(R11)

    C21 = Conv1D(filters=32, kernel_size=2, activation='relu', padding='same')(M11)
    C22 = Conv1D(filters=32, kernel_size=2, padding='same')(C21)
    A21 = Add()([M11, C22])
    R21 = Activation(activation='relu')(A21)
    M21 = MaxPool1D(pool_size=2, strides=2)(R21)

    C31 = Conv1D(filters=32, kernel_size=2, activation='relu', padding='same')(M21)
    C32 = Conv1D(filters=32, kernel_size=2, padding='same')(C31)
    A31 = Add()([M21, C32])
    R31 = Activation(activation='relu')(A31)
    M31 = MaxPool1D(pool_size=2, strides=2)(R31)

    C41 = Conv1D(filters=32, kernel_size=2, activation='relu', padding='same')(M31)
    C42 = Conv1D(filters=32, kernel_size=2, padding='same')(C41)
    A41 = Add()([M31, C42])
    R41 = Activation(activation='relu')(A41)
    M41 = MaxPool1D(pool_size=2, strides=2)(R41)

    C51 = Conv1D(filters=32, kernel_size=2, activation='relu', padding='same')(M41)
    C52 = Conv1D(filters=32, kernel_size=2, padding='same')(C51)
    A51 = Add()([M41, C52])
    R51 = Activation(activation='relu')(A51)
    M51 = MaxPool1D(pool_size=2, strides=2)(R51)

    F1 = Flatten()(M51)
    D1 = Dense(32)(F1)
    R1 = Activation(activation='relu')(D1)
    D2 = Dense(32)(R1)
    D3 = Dense(final_layer_size)(D2)

    O = Activation(activation='sigmoid')(D3)

    return Model(inputs=I, outputs=O)

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(0.00001, decay_steps=10000, decay_rate=0.75)
adam = Adam(learning_rate=lr_schedule, beta_1=0.9, beta_2=0.999, amsgrad=False)

In [ ]:
n_classes = len(np.unique(y2))
print(n_classes)
final_layer = 1
model_2 = make_model(final_layer)
model_2.summary()

In [ ]:
model_2.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss', patience=8),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

history_2 = model_2.fit(np.expand_dims(train_x2, axis=2), 
                          train_y2, 
                          validation_split=0.20,
                          callbacks=callbacks,  
                          epochs=30,
                          batch_size=8,
                          verbose=1)



In [ ]:
# Train data
history_1 = model_2.fit(np.expand_dims(train_x, axis=2), 
                           train_y, 
                           validation_split=0.15,
                           epochs=30,
                           batch_size=256,
                           verbose=0)

In [ ]:
def plot_learning(history):
    plt.subplot(211)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.legend(["accuracy", 'val_accuracy'])
    plt.subplot(212)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'], label = "val_loss")
    plt.legend(["loss", "val_loss"])
    plt.show()

In [ ]:
plot_learning(history_1)

In [ ]:
model.load_weights('best_model.h5')
plot_learning(history_2)

In [ ]:
# Results Train Data
result_1 = model_2.evaluate(np.expand_dims(val_x, axis=2), val_y, batch_size=128)
print(f"The accuracy on the testing set is {np.round(result_1[1]*100,1)}%")

In [ ]:
# Results Test Data

result_2 = model_2.evaluate(np.expand_dims(val_x2, axis=2), val_y2, batch_size=128)
print(f"The accuracy on the testing set is {np.round(result_2[1]*100,1)}%")

In [ ]:
# Transfer Learning


D1 = Dense(32)(model2.output)
D2 = Dense(32)(D1)
O = Dense(1, activation='sigmoid')(D2)
model_transfer = Model(inputs=model2.input, outputs=O)

for layer in model_transfer.layers[:-3]:
    # layer.trainable = False
    layer.trainable = True

for layer in model_transfer.layers[-3:]:
    layer.trainable = True
    
model_transfer.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
callbacks = [EarlyStopping(monitor='val_loss', patience=8),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]


history_transfer = model_transfer.fit(np.expand_dims(train_x2, axis=2), 
                    train_y2, 
                    validation_split=0.2,
                    epochs=30,
                    callbacks = callbacks,
                    batch_size=8,
                    verbose=1)

In [ ]:
model_transfer.load_weights('best_model.h5')
plot_learning(history_transfer)

In [ ]:
print("Trainability of the layers \n")
for layer in model_transfer.layers:
    config = layer.get_config()
    print(f"{config['name']} : {config.get('trainable')}")

In [ ]:
# Train Without Bottleneck

D1 = Dense(32)(model_2.layers[-3].output)
D2 = Dense(32)(D1)
O = Dense(1, activation='sigmoid')(D2)
model_transfer = Model(inputs=model_2.input, outputs=O)


for layer in model_transfer.layers[:-3]:
    layer.trainable = False

for layer in model_transfer.layers[-3:]:
    layer.trainable = True
    
model_transfer.compile(optimizer=adam, loss='mse', metrics=['accuracy'])
callbacks = [EarlyStopping(monitor='val_loss', patience=8),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]
history_transfer = model_transfer.fit(np.expand_dims(train_x2, axis=2), 
                    train_y2, 
                    validation_split=0.20,
                    epochs=50,
                    callbacks = callbacks,
                    batch_size=8,
                    verbose=1)

plot_learning(history_transfer)

In [ ]:
# Without bottle neck and whole trainable model

D1 = Dense(32)(model_2.layers[-3].output)
D2 = Dense(32)(D1)
O = Dense(1, activation='sigmoid')(D2)
model_transfer = Model(inputs=model_2.input, outputs=O)
model_transfer.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
history_transfer = model_transfer.fit(np.expand_dims(train_x, axis=2), 
                    train_y, 
                    validation_split=0.15,
                    epochs=100,
                    batch_size=8,
                    verbose=0)
plot_learning(history_transfer)

In [ ]:
# Transfer Learning with test data


D1 = Dense(32)(model_2.output)
D2 = Dense(32)(D1)
O = Dense(1, activation='sigmoid')(D2)
model_transfer2 = Model(inputs=model_2.input, outputs=O)

for layer in model_transfer2.layers[:-3]:
    layer.trainable = False

for layer in model_transfer2.layers[-3:]:
    layer.trainable = True
    
model_transfer2.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])


history_transfer2 = model_transfer2.fit(np.expand_dims(train_x2, axis=2), 
                    train_y2, 
                    validation_split=0.15,
                    epochs=30,
                    batch_size=8,
                    verbose=0)

In [ ]:
plot_learning(history_transfer2)

In [ ]:
# Test Without Bottleneck

D1 = Dense(32)(model_2.layers[-3].output)
D2 = Dense(32)(D1)
O = Dense(1, activation='sigmoid')(D2)
model_transfer2 = Model(inputs=model_2.input, outputs=O)


for layer in model_transfer2.layers[:-3]:
    layer.trainable = False

for layer in model_transfer2.layers[-3:]:
    layer.trainable = True
    
model_transfer2.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
history_transfer2 = model_transfer2.fit(np.expand_dims(train_x2, axis=2), 
                    train_y2, 
                    validation_split=0.15,
                    epochs=50,
                    batch_size=8,
                    verbose=1)

plot_learning(history_transfer2)

In [ ]:
# Test without bottleneck and all trainable

D1 = Dense(32)(model_2.layers[-3].output)
D2 = Dense(32)(D1)
O = Dense(1, activation='softmax')(D2)
model_transfer2 = Model(inputs=model_2.input, outputs=O)
model_transfer2.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
history_transfer2 = model_transfer2.fit(np.expand_dims(train_x2, axis=2), 
                    train_y2, 
                    validation_split=0.15,
                    epochs=50,
                    batch_size=8,
                    verbose=1)
plot_learning(history_transfer2)

In [ ]:
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
loocv = LeaveOneOut()
results = cross_val_score(GB, rep_x2, rep_y2, cv = loocv)
print("Accuracy: %.3f%% (%.3f%%)") % (results.mean()*100.0, results.std()*100.0)

In [ ]:
# sensitivity analysis of k in k-fold cross-validation
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot
# Statistical test
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from numpy import std
from numpy import mean
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import log
from sklearn.ensemble import GradientBoostingClassifier


# evaluate the model using a given test condition
def evaluate_model(cv):
	# get the dataset
	# get the model
	RF = RandomForestClassifier() 
	# evaluate the model
	scores = cross_val_score(RF, rep_xL, rep_yL, scoring='accuracy', cv=cv, n_jobs=-1)
	# return scores
	return mean(scores), scores.min(), scores.max()



ideal, _, _ = evaluate_model(LeaveOneOut())
print('Ideal: %0.3f' % ideal)



# record mean and min/max of each set of results
means = []
mins = []
maxs = []

folds = range(2,31)
# evaluate each k value

for k in folds:
	# define the test condition
	cv = KFold(n_splits=k, shuffle=True, random_state=1)
	# evaluate k value
	k_mean, k_min, k_max = evaluate_model(cv)
	# report performance
	print('> folds=%d, accuracy=%.3f (%.3f,%.3f)' % (k, k_mean, k_min, k_max))
	# store mean accuracy
	means.append(k_mean)
	# store min and max relative to the mean
	mins.append(k_mean - k_min)
	maxs.append(k_max - k_mean)
 

# line plot of k mean values with min/max error bars
pyplot.errorbar(folds, means, yerr=[mins, maxs], fmt='o')
# plot the ideal case in a separate color
pyplot.plot(folds, [ideal for _ in range(len(folds))], color='r')
# show the plot
pyplot.title('Cross Validation Acc per Folds')
pyplot.xlabel('Folds')
pyplot.ylabel('Accuracy')
pyplot.show()



In [ ]:
# LOOCV vs CV
# correlation between test harness and ideal test condition
from numpy import mean
from numpy import isnan
from numpy import asarray
from numpy import polyfit
from scipy.stats import pearsonr
from matplotlib import pyplot
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


# get a list of models to evaluate
def get_models():
	models = []
	models.append(LogisticRegression())
	models.append(RidgeClassifier())
	models.append(SGDClassifier())
	models.append(PassiveAggressiveClassifier())
	models.append(KNeighborsClassifier())
	models.append(DecisionTreeClassifier())
	models.append(ExtraTreeClassifier())
	models.append(LinearSVC())
	models.append(SVC())
	models.append(GaussianNB())
	models.append(AdaBoostClassifier())
	models.append(BaggingClassifier())
	models.append(RandomForestClassifier())
	models.append(ExtraTreesClassifier())
	models.append(GaussianProcessClassifier())
	models.append(GradientBoostingClassifier())
	models.append(LinearDiscriminantAnalysis())
	models.append(QuadraticDiscriminantAnalysis())
	return models

# evaluate the model using a given test condition
def evaluate_model(cv, model):
	# get the dataset
	# evaluate the model
	scores = cross_val_score(model, rep_xL, rep_yL, scoring='accuracy', cv=cv, n_jobs=-1)
	# return scores
	return mean(scores)

# define test conditions
ideal_cv = LeaveOneOut()
cv = KFold(n_splits=10, shuffle=True, random_state=1)
# get the list of models to consider
models = get_models()
# collect results
ideal_results, cv_results = [], []
# evaluate each model
for model in models:
	# evaluate model using each test condition
	cv_mean = evaluate_model(cv, model)
	ideal_mean = evaluate_model(ideal_cv, model)
	# check for invalid results
	if isnan(cv_mean) or isnan(ideal_mean):
		continue
	# store results
	cv_results.append(cv_mean)
	ideal_results.append(ideal_mean)
	# summarize progress
	print('>%s: ideal=%.3f, cv=%.3f' % (type(model).__name__, ideal_mean, cv_mean))
# calculate the correlation between each test condition
corr, _ = pearsonr(cv_results, ideal_results)
print('Correlation: %.3f' % corr)
# scatter plot of results
pyplot.scatter(cv_results, ideal_results)
# plot the line of best fit
coeff, bias = polyfit(cv_results, ideal_results, 1)
line = coeff * asarray(cv_results) + bias
pyplot.plot(cv_results, line, color='r')
# label the plot
pyplot.title('10-fold CV vs LOOCV Mean Accuracy')
pyplot.xlabel('Mean Accuracy (10-fold CV)')
pyplot.ylabel('Mean Accuracy (LOOCV)')
# show the plot
pyplot.show()
# correlation between test harness and ideal test condition


In [ ]:
# Leave one out cross validation for deep learning algorithms
import numpy as np
from sklearn.model_selection import LeaveOneOut

# train_x2, val_x2, train_y2, val_y2
# rep_x2, rep_y2
loo = LeaveOneOut()

for train_index, test_index in loo.split(rep_x2):
  train_x2, val_x2 = rep_x2[train_index], rep_x2[test_index]
  train_y2, val_y2 = rep_y2[train_index], rep_y2[test_index]
  
  D1 = Dense(32)(model_vgg.output)
  D2 = Dense(32)(D1)
  O = Dense(1, activation='sigmoid')(D2)
  modelvgg_transfer = Model(inputs=model_vgg.input, outputs=O)

  for layer in modelvgg_transfer.layers[:-3]:
    # layer.trainable = False
    layer.trainable = True

  for layer in modelvgg_transfer.layers[-3:]:
    layer.trainable = True
    
  modelvgg_transfer.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
  callbacks = [EarlyStopping(monitor='val_loss', patience=8),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]


  history_transfer = modelvgg_transfer.fit(np.expand_dims(train_x2, axis=2), 
                    train_y2, 
                    validation_split=0.2,
                    epochs=30,
                    callbacks = callbacks,
                    batch_size=8,
                    verbose=1)



In [ ]:
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
import numpy as np

kfold = KFold(n_splits = 10, shuffle=True)

# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []
# K-fold Cross Validation model evaluation
fold_no = 1

for train, test in kfold.split(rep_x2, rep_y2):
  train_x2, val_x2 = rep_x2[train], rep_x2[test]
  train_y2, val_y2 = rep_y2[train], rep_y2[test]

  D1 = Dense(32)(model_vgg.output)
  D2 = Dense(32)(D1)
  O = Dense(1, activation='sigmoid')(D2)
  modelvgg_transfer = Model(inputs=model_vgg.input, outputs=O)

  for layer in modelvgg_transfer.layers[:-3]:
    # layer.trainable = False
    layer.trainable = True

  for layer in modelvgg_transfer.layers[-3:]:
    layer.trainable = True
    
  modelvgg_transfer.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
  callbacks = [EarlyStopping(monitor='val_loss', patience=8),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  history_transfer = modelvgg_transfer.fit(np.expand_dims(train_x2, axis=2), 
                    train_y2, 
                    validation_split=0.2,
                    epochs=30,
                    callbacks = callbacks,
                    batch_size=8,
                    verbose=1)
  
  scores = modelvgg_transfer.evaluate(val_x2, val_y2, verbose = 1)

  print(f'Score for fold {fold_no}: {modelvgg_transfer.metrics_names[0]} of {scores[0]}; {modelvgg_transfer.metrics_names[1]} of {scores[1]*100}%')



  
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1


# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')